In [25]:
#####다시 해보자구~~~!!도전!!! >o< ^o^
###T-Td 함수 추가해보기(GHI함수 빼기)

import pandas as pd
import numpy as np
import os
import glob
import random
import warnings
import tensorflow.keras.backend as K
warnings.filterwarnings("ignore")

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.models import Sequential, load_model,Model
from tensorflow.keras.layers import Dense, Conv1D, Dropout, MaxPool1D,Flatten, Reshape, LSTM,Input,concatenate,BatchNormalization
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau
from lightgbm import LGBMRegressor
from tensorflow.keras.optimizers import Adam


# 파일 불러오기
train = pd.read_csv('./train/train.csv')
submission = pd.read_csv('sample_submission.csv')

#1. DATA

# GHI column 추가 함수
def Add_features(data):
    c = 243.12
    b = 17.62
    gamma = (b * (data['T']) / (c + (data['T']))) + np.log(data['RH'] / 100)
    dp = ( c * gamma) / (b - gamma)
    data.insert(1,'Td',dp)
    data.insert(1,'T-Td',data['T']-data['Td'])
    data.insert(1,'GHI',data['DNI']+data['DHI'])

    sunup=[]
    for i in range(len(data)):
        temp = data.iloc[i,-1]
        if temp>0.0:
            sunup.append(1)
        else :
            sunup.append(0)
    data['Sun_up']=sunup

    # Time 추가
    Time=[]
    for i in range(len(data)):
        temp = 60*data.iloc[i,1]+data.iloc[i,2]
        Time.append(temp)
    data['Time']=Time


    return data

# train data column정의
def preprocess_data(data, is_train=True):
    c = 243.12
    b = 17.62
    gamma = (b * (data['T']) / (c + (data['T']))) + np.log(data['RH'] / 100)
    dp = ( c * gamma) / (b - gamma)
    data.insert(1,'Td',dp)
    data.insert(1,'T-Td',data['T']-data['Td'])
    data.insert(1,'GHI',data['DNI']+data['DHI'])
    sunup=[]
    for i in range(len(data)):
        temp = data.iloc[i,-1]
        if temp>0.0:
            sunup.append(1)
        else :
            sunup.append(0)
    data['Sun_up']=sunup
    # Time 추가
    Time=[]
    for i in range(len(data)):
        temp = 60*data.iloc[i,1]+data.iloc[i,2]
        Time.append(temp)
    data['Time']=Time
    temp = data.copy()
    temp = temp[['Hour','TARGET','T-Td','DHI','DNI','WS','RH','T','Time','Sun_up']]

    if is_train==True:          
        temp['Target1'] = temp['TARGET'].shift(-48).fillna(method='ffill')   # 다음날의 Target
        temp['Target2'] = temp['TARGET'].shift(-48*2).fillna(method='ffill') # 다다음날의 Target
        temp = temp.dropna()    # 결측값 제거
        return temp.iloc[:-96]  # 뒤에서 이틀은 뺀다. (예측하고자 하는 날짜이기 때문)

    elif is_train == False:
        temp = temp[['Hour','TARGET','T-Td','DHI','DNI','WS','RH','T','Time','Sun_up']]

        return temp.iloc[-48:, :]

df_train = preprocess_data(train)
# print(df_train.shape)   # (52464, 10)
# print(df_train[:48])

#-------------test 함수정의
df_test = []
for i in range(81):
    file_path = 'test/' + str(i) + '.csv'
    temp = pd.read_csv(file_path)
    temp = preprocess_data(temp, is_train=False)
    df_test.append(temp)

df_test = pd.concat(df_test)
test = np.array(df_test)
print(df_test.shape) # (3888, 8)
x_test = df_test.to_numpy()

###=====================================================================
# 시계열 데이터(함수) --> y1, y2 로 나눠 각각 훈련시키기
def split_xy(dataset, time_steps) :
    x, y1, y2 = [],[],[]
    for i in range(len(dataset)) :
        x_end = i + time_steps
        if x_end > len(dataset) :
            break
        tmp_x = dataset[i:x_end, :-2] # ['Hour' ~ 'T']
        tmp_y1 = dataset[x_end-1:x_end,-2] # Target1
        tmp_y2 = dataset[x_end-1:x_end,-1]   # Target2
        x.append(tmp_x)
        y1.append(tmp_y1)
        y2.append(tmp_y2)
    return np.array(x), np.array(y1), np.array(y2)

X = df_train.to_numpy()
# print(X.shape)      # (52464, 10)
x,y1,y2 = split_xy(X,1)
print(x.shape, y1.shape, y2.shape) #(52464, 1, 8) (52464, 1) (52464, 1)

#####===========전처리
x_train, x_val, y1_train, y1_val, y2_train, y2_val = train_test_split(x,y1,y2,
                           train_size = 0.7,shuffle = False, random_state = 0)

# print(x_train.shape,x_val.shape) #(36724, 1, 8) (15740, 1, 8)
print(y1_train.shape, y1_val.shape, y2_train.shape, y2_val.shape) #(36724, 1) (15740, 1) (36724, 1) (15740, 1)
x_train= x_train.reshape(36724*1, 10)
x_val= x_val.reshape(15740*1, 10)

scaler = StandardScaler()
scaler.fit(x_train)
x_train = scaler.transform(x_train)
x_test = scaler.transform(x_test)
x_val = scaler.transform(x_val)

x_train= x_train.reshape(36724, 1, 10)
x_val= x_val.reshape(15740, 1, 10)
x_test= x_test.reshape(3888, 1, 10)
# print(x_train.shape,x_val.shape,x_test.shape) #(36724, 1, 8) (15740, 1, 8) (3888, 1, 8)

# 함수 : Quantile loss definition
def quantile_loss(q, y_true, y_pred):
    err = (y_true - y_pred)
    return K.mean(K.maximum(q*err, (q-1)*err), axis=-1)

q = [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9]

#2. 모델링

def modeling() :
    inputs = Input(shape=(x_train.shape[1], x_train.shape[2]))
    adam = Adam(learning_rate=0.003)
    conv = Conv1D(filters = 256,kernel_size=2,activation='relu',padding='same')(inputs)
    conv = BatchNormalization()(conv)
    conv = Conv1D(filters = 128,kernel_size=2,activation='relu',padding='same')(conv)
    conv = BatchNormalization()(conv)
    conv = Conv1D(filters = 128,kernel_size=2,activation='relu',padding='same')(conv)
    conv = BatchNormalization()(conv)
    conv = Conv1D(filters = 64,kernel_size=2,activation='relu',padding='same')(conv)
    conv = BatchNormalization()(conv)
    conv = Conv1D(filters = 32,kernel_size=2,activation='relu',padding='same')(conv)
    conv = BatchNormalization()(conv)
    conv = Flatten()(conv)

    conv1 = Conv1D(filters = 256,kernel_size=2,activation='relu',padding='same')(inputs)
    conv1 = BatchNormalization()(conv1)
    conv1 = Conv1D(filters = 128,kernel_size=2,activation='relu',padding='same')(conv1)
    conv1 = BatchNormalization()(conv1)
    conv1 = Conv1D(filters = 128,kernel_size=2,activation='relu',padding='same')(conv1)
    conv1 = BatchNormalization()(conv1)
    conv1 = Conv1D(filters = 64,kernel_size=2,activation='relu',padding='same')(conv1)
    conv1 = BatchNormalization()(conv1)
    conv1 = Conv1D(filters = 32,kernel_size=2,activation='relu',padding='same')(conv1)
    conv1 = BatchNormalization()(conv1)
    conv1 = Flatten()(conv1)

    conv2 = Conv1D(filters = 256,kernel_size=3,activation='relu',padding='same')(inputs)
    conv2 = BatchNormalization()(conv2)
    conv2 = Conv1D(filters = 128,kernel_size=3,activation='relu',padding='same')(conv2)
    conv2 = BatchNormalization()(conv2)
    conv2 = Conv1D(filters = 128,kernel_size=3,activation='relu',padding='same')(conv2)
    conv2 = BatchNormalization()(conv2)
    conv2 = Conv1D(filters = 64,kernel_size=3,activation='relu',padding='same')(conv2)
    conv2 = BatchNormalization()(conv2)
    conv2 = Conv1D(filters = 32,kernel_size=3,activation='relu',padding='same')(conv2)
    conv2 = BatchNormalization()(conv2)
    conv2 = Flatten()(conv2)
    
    conv3 = Conv1D(filters = 256,kernel_size=1,activation='relu',padding='same')(inputs)
    conv3 = BatchNormalization()(conv3)
    conv3 = Conv1D(filters = 128,kernel_size=1,activation='relu',padding='same')(conv3)
    conv3 = BatchNormalization()(conv3)
    conv3 = Conv1D(filters = 128,kernel_size=1,activation='relu',padding='same')(conv3)
    conv3 = BatchNormalization()(conv3)
    conv3 = Conv1D(filters = 64,kernel_size=1,activation='relu',padding='same')(conv3)
    conv3 = BatchNormalization()(conv3)
    conv3 = Conv1D(filters = 32,kernel_size=1,activation='relu',padding='same')(conv3)
    conv3 = BatchNormalization()(conv3)
    conv3 = Flatten()(conv3)
    
    lstm = GRU(256,activation='relu')(inputs)
    lstm = LayerNormalization()(lstm)
    concat = concatenate([conv,conv1,conv2,conv3,lstm])
    dense = Dense(128,activation='relu')(concat)
    dense = Dense(64,activation='relu')(dense)
    dense = Dense(32,activation='relu')(dense)
    dense = Dense(8,activation='relu')(dense)
    outputs = Dense(1,activation='relu')(dense)
    model = Model(inputs=inputs,outputs=outputs)
    return model
# 3. 컴파일, 훈련
#####========컴파일,  훈련, predict
#y1, y2 각각 저장하기

from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau, ModelCheckpoint
es = EarlyStopping(monitor = 'val_loss', patience = 30)
lr = ReduceLROnPlateau(monitor = 'val_loss', patience = 20, factor = 0.9, verbose = 1)
epochs = 500
bs = 16

# 내일!!
# x = []
for i in q:
    if q!=0.5:
        print("\n\n ",i,"\n\n")
        model = modeling()
        filepath_cp = f'models2/dacon_y1_quantile_{i:.1f}.hdf5'
        cp = ModelCheckpoint(filepath_cp,save_best_only=True,monitor = 'val_loss')
        model.compile(loss = lambda y_true,y_pred: quantile_loss(i,y_true,y_pred), optimizer = 'adam', metrics = [lambda y,y_pred: quantile_loss(i,y,y_pred)])
        model.fit(x_train,y1_train,epochs = epochs, batch_size = bs, validation_data = (x_val,y1_val),callbacks = [es,cp,lr])
        pred = pd.DataFrame(model.predict(x_test).reshape(3888,1).round(2))
        x.append(pred)
    else:
        print("\n\n ",i,"\n\n")
        model = modeling()
        filepath_cp = f'models2/dacon_y1_quantile_{i:.1f}.hdf5'
        cp = ModelCheckpoint(filepath_cp,save_best_only=True,monitor = 'val_loss')
        model.compile(loss = 'mae', optimizer = 'adam')
        model.fit(x_train,y1_train,epochs = epochs, batch_size = bs, validation_data = (x_val,y1_val),callbacks = [es,cp,lr])
        pred = pd.DataFrame(model.predict(x_test).reshape(3888,1).round(2))
        x.append(pred)
df_temp1 = pd.concat(x, axis = 1)
df_temp1[df_temp1<0] = 0
num_temp1 = df_temp1.to_numpy()
submission.loc[submission.id.str.contains("Day7"), "q_0.1":] = num_temp1

x = []
# 모레!!
for i in [0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.9]:
    if q!=0.5:
        print("\n\n ",i,"\n\n")
        model = modeling()
        filepath_cp = f'models2/dacon_y2_quantile_{i:.1f}.hdf5'
        cp = ModelCheckpoint(filepath_cp,save_best_only=True,monitor = 'val_loss')
        model.compile(loss = lambda y_true,y_pred: quantile_loss(i,y_true,y_pred), optimizer = 'adam', metrics = [lambda y,y_pred: quantile_loss(i,y,y_pred)])
        model.fit(x_train,y2_train,epochs = epochs, batch_size = bs, validation_data = (x_val,y2_val),callbacks = [es,cp,lr])
        pred = pd.DataFrame(model.predict(x_test).reshape(3888,1).round(2))
        x.append(pred)
    else:
        print("\n\n ",i,"\n\n")
        model = modeling()
        filepath_cp = f'models2/dacon_y2_quantile_{i:.1f}.hdf5'
        cp = ModelCheckpoint(filepath_cp,save_best_only=True,monitor = 'val_loss')
        model.compile(loss = 'mae', optimizer = 'adam')
        model.fit(x_train,y2_train,epochs = epochs, batch_size = bs, validation_data = (x_val,y2_val),callbacks = [es,cp,lr])
        pred = pd.DataFrame(model.predict(x_test).reshape(3888,1).round(2))
        x.append(pred)
df_temp2 = pd.concat(x, axis = 1)
df_temp2[df_temp2<0] = 0
num_temp2 = df_temp2.to_numpy()
submission.loc[submission.id.str.contains("Day8"), "q_0.1":] = num_temp2

##??????????????근데 이 지표가 잘됐다는거 평가하는 방법있을까?????????  로스밖에 없어
submission.to_csv('models2/upgrade_sub_0125_돌려놓고감_2.csv', index = False)
##제출후 등수 135  -- 1.94653

(3888, 10)
(52464, 1, 10) (52464, 1) (52464, 1)
(36724, 1) (15740, 1) (36724, 1) (15740, 1)


  0.1 


Epoch 1/500
2296/2296 [==============================] - 37s 16ms/step - loss: 1.4445 - <lambda>: 1.4445 - val_loss: 1.5479 - val_<lambda>: 1.5479
Epoch 2/500
2296/2296 [==============================] - 36s 16ms/step - loss: 1.4126 - <lambda>: 1.4126 - val_loss: 1.5175 - val_<lambda>: 1.5175
Epoch 3/500
2296/2296 [==============================] - 36s 16ms/step - loss: 1.4050 - <lambda>: 1.4050 - val_loss: 1.5490 - val_<lambda>: 1.5490
Epoch 4/500
2296/2296 [==============================] - 35s 15ms/step - loss: 1.3986 - <lambda>: 1.3986 - val_loss: 1.5353 - val_<lambda>: 1.5353
Epoch 5/500
2296/2296 [==============================] - 35s 15ms/step - loss: 1.3871 - <lambda>: 1.3871 - val_loss: 1.6655 - val_<lambda>: 1.6655
Epoch 6/500
2296/2296 [==============================] - 35s 15ms/step - loss: 1.3839 - <lambda>: 1.3839 - val_loss: 1.5304 - val_<lambda>: 1.5304
Epoch 7/500
229

2296/2296 [==============================] - 35s 15ms/step - loss: 2.2077 - <lambda>: 2.2077 - val_loss: 2.5348 - val_<lambda>: 2.5348
Epoch 12/500
2296/2296 [==============================] - 35s 15ms/step - loss: 2.2003 - <lambda>: 2.2003 - val_loss: 2.5000 - val_<lambda>: 2.5000
Epoch 13/500
2296/2296 [==============================] - 35s 15ms/step - loss: 2.1910 - <lambda>: 2.1910 - val_loss: 2.5142 - val_<lambda>: 2.5142
Epoch 14/500
2296/2296 [==============================] - 35s 15ms/step - loss: 2.1797 - <lambda>: 2.1797 - val_loss: 2.5137 - val_<lambda>: 2.5137
Epoch 15/500
2296/2296 [==============================] - 35s 15ms/step - loss: 2.1754 - <lambda>: 2.1754 - val_loss: 2.5550 - val_<lambda>: 2.5550
Epoch 16/500
2296/2296 [==============================] - 35s 15ms/step - loss: 2.1786 - <lambda>: 2.1786 - val_loss: 2.4702 - val_<lambda>: 2.4702
Epoch 17/500
2296/2296 [==============================] - 35s 15ms/step - loss: 2.1605 - <lambda>: 2.1605 - val_loss: 2.5277 

2296/2296 [==============================] - 35s 15ms/step - loss: 2.4561 - <lambda>: 2.4561 - val_loss: 3.0756 - val_<lambda>: 3.0756
Epoch 32/500
2296/2296 [==============================] - 35s 15ms/step - loss: 2.4487 - <lambda>: 2.4487 - val_loss: 3.1318 - val_<lambda>: 3.1318
Epoch 33/500
2296/2296 [==============================] - 35s 15ms/step - loss: 2.4350 - <lambda>: 2.4350 - val_loss: 3.0932 - val_<lambda>: 3.0932
Epoch 34/500
2296/2296 [==============================] - 35s 15ms/step - loss: 2.4263 - <lambda>: 2.4263 - val_loss: 3.0818 - val_<lambda>: 3.0818


  0.4 


Epoch 1/500
2296/2296 [==============================] - 35s 15ms/step - loss: 3.1310 - <lambda>: 3.1310 - val_loss: 3.3535 - val_<lambda>: 3.3535
Epoch 2/500
2296/2296 [==============================] - 35s 15ms/step - loss: 2.9611 - <lambda>: 2.9611 - val_loss: 3.2803 - val_<lambda>: 3.2803
Epoch 3/500
2296/2296 [==============================] - 35s 15ms/step - loss: 2.9055 - <lambda>: 2.9055 - val_loss:

Epoch 13/500
2296/2296 [==============================] - 35s 15ms/step - loss: 2.6801 - <lambda>: 2.6801 - val_loss: 3.0087 - val_<lambda>: 3.0087
Epoch 14/500
2296/2296 [==============================] - 35s 15ms/step - loss: 2.6719 - <lambda>: 2.6719 - val_loss: 2.9955 - val_<lambda>: 2.9955
Epoch 15/500
2296/2296 [==============================] - 35s 15ms/step - loss: 2.6643 - <lambda>: 2.6643 - val_loss: 3.0585 - val_<lambda>: 3.0585
Epoch 16/500
2296/2296 [==============================] - 35s 15ms/step - loss: 2.6531 - <lambda>: 2.6531 - val_loss: 3.0735 - val_<lambda>: 3.0735
Epoch 17/500
2296/2296 [==============================] - 35s 15ms/step - loss: 2.6525 - <lambda>: 2.6525 - val_loss: 2.9867 - val_<lambda>: 2.9867
Epoch 18/500
2296/2296 [==============================] - 35s 15ms/step - loss: 2.6378 - <lambda>: 2.6378 - val_loss: 2.9931 - val_<lambda>: 2.9931
Epoch 19/500
2296/2296 [==============================] - 35s 15ms/step - loss: 2.6333 - <lambda>: 2.6333 - val_

2296/2296 [==============================] - 35s 15ms/step - loss: 2.3069 - <lambda>: 2.3069 - val_loss: 2.7001 - val_<lambda>: 2.7001
Epoch 26/500
2296/2296 [==============================] - 35s 15ms/step - loss: 2.2979 - <lambda>: 2.2979 - val_loss: 2.7456 - val_<lambda>: 2.7456
Epoch 27/500
2296/2296 [==============================] - 35s 15ms/step - loss: 2.2919 - <lambda>: 2.2919 - val_loss: 2.7322 - val_<lambda>: 2.7322
Epoch 28/500
2296/2296 [==============================] - 35s 15ms/step - loss: 2.2943 - <lambda>: 2.2943 - val_loss: 2.7319 - val_<lambda>: 2.7319
Epoch 29/500
2296/2296 [==============================] - 35s 15ms/step - loss: 2.2829 - <lambda>: 2.2829 - val_loss: 2.7252 - val_<lambda>: 2.7252
Epoch 30/500
2295/2296 [============================>.] - ETA: 0s - loss: 2.2793 - <lambda>: 2.2793
Epoch 00030: ReduceLROnPlateau reducing learning rate to 0.0009000000427477062.
2296/2296 [==============================] - 35s 15ms/step - loss: 2.2792 - <lambda>: 2.2792 

Epoch 4/500
2296/2296 [==============================] - 35s 15ms/step - loss: 0.8861 - <lambda>: 0.8861 - val_loss: 0.9712 - val_<lambda>: 0.9712
Epoch 5/500
2296/2296 [==============================] - 35s 15ms/step - loss: 0.8677 - <lambda>: 0.8677 - val_loss: 0.9468 - val_<lambda>: 0.9468
Epoch 6/500
2296/2296 [==============================] - 35s 15ms/step - loss: 0.8521 - <lambda>: 0.8521 - val_loss: 0.9640 - val_<lambda>: 0.9640
Epoch 7/500
2296/2296 [==============================] - 35s 15ms/step - loss: 0.8585 - <lambda>: 0.8585 - val_loss: 0.9001 - val_<lambda>: 0.9001
Epoch 8/500
2296/2296 [==============================] - 35s 15ms/step - loss: 0.8379 - <lambda>: 0.8379 - val_loss: 0.9089 - val_<lambda>: 0.9089
Epoch 9/500
2296/2296 [==============================] - 35s 15ms/step - loss: 0.8309 - <lambda>: 0.8309 - val_loss: 0.9308 - val_<lambda>: 0.9308
Epoch 10/500
2296/2296 [==============================] - 35s 15ms/step - loss: 0.8328 - <lambda>: 0.8328 - val_loss: 

2296/2296 [==============================] - 35s 15ms/step - loss: 0.7438 - <lambda>: 0.7438 - val_loss: 0.9229 - val_<lambda>: 0.9229
Epoch 60/500
2296/2296 [==============================] - 35s 15ms/step - loss: 0.7442 - <lambda>: 0.7442 - val_loss: 0.9274 - val_<lambda>: 0.9274
Epoch 61/500
2296/2296 [==============================] - 35s 15ms/step - loss: 0.7410 - <lambda>: 0.7410 - val_loss: 0.9350 - val_<lambda>: 0.9350
Epoch 62/500
2296/2296 [==============================] - 35s 15ms/step - loss: 0.7383 - <lambda>: 0.7383 - val_loss: 0.8773 - val_<lambda>: 0.8773
Epoch 63/500
2296/2296 [==============================] - 35s 15ms/step - loss: 0.7376 - <lambda>: 0.7376 - val_loss: 0.8781 - val_<lambda>: 0.8781
Epoch 64/500
2296/2296 [==============================] - 35s 15ms/step - loss: 0.7377 - <lambda>: 0.7377 - val_loss: 0.9101 - val_<lambda>: 0.9101
Epoch 65/500
2296/2296 [==============================] - 38s 17ms/step - loss: 0.7375 - <lambda>: 0.7375 - val_loss: 0.9113 

ValueError: Must have equal len keys and value when setting with an ndarray

In [26]:
#####다시 해보자구~~~!!도전!!! >o< ^o^
###T-Td 함수 추가해보기(GHI함수 빼기)

import pandas as pd
import numpy as np
import os
import glob
import random
import warnings
import tensorflow.keras.backend as K
warnings.filterwarnings("ignore")

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.models import Sequential, load_model,Model
from tensorflow.keras.layers import Dense, Conv1D, Dropout, MaxPool1D,Flatten, Reshape, LSTM,Input,concatenate,BatchNormalization,LayerNormalization,GRU
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau
from lightgbm import LGBMRegressor
from tensorflow.keras.optimizers import Adam


# 파일 불러오기
train = pd.read_csv('./train/train.csv')
submission = pd.read_csv('sample_submission.csv')

#1. DATA

# GHI column 추가 함수
def Add_features(data):
    c = 243.12
    b = 17.62
    gamma = (b * (data['T']) / (c + (data['T']))) + np.log(data['RH'] / 100)
    dp = ( c * gamma) / (b - gamma)
    data.insert(1,'Td',dp)
    data.insert(1,'T-Td',data['T']-data['Td'])
    data.insert(1,'GHI',data['DNI']+data['DHI'])

    sunup=[]
    for i in range(len(data)):
        temp = data.iloc[i,-1]
        if temp>0.0:
            sunup.append(1)
        else :
            sunup.append(0)
    data['Sun_up']=sunup

    # Time 추가
    Time=[]
    for i in range(len(data)):
        temp = 60*data.iloc[i,1]+data.iloc[i,2]
        Time.append(temp)
    data['Time']=Time


    return data

# train data column정의
def preprocess_data(data, is_train=True):
    c = 243.12
    b = 17.62
    gamma = (b * (data['T']) / (c + (data['T']))) + np.log(data['RH'] / 100)
    dp = ( c * gamma) / (b - gamma)
    data.insert(1,'Td',dp)
    data.insert(1,'T-Td',data['T']-data['Td'])
    data.insert(1,'GHI',data['DNI']+data['DHI'])
    sunup=[]
    for i in range(len(data)):
        temp = data.iloc[i,-1]
        if temp>0.0:
            sunup.append(1)
        else :
            sunup.append(0)
    data['Sun_up']=sunup
    # Time 추가
    Time=[]
    for i in range(len(data)):
        temp = 60*data.iloc[i,1]+data.iloc[i,2]
        Time.append(temp)
    data['Time']=Time
    temp = data.copy()
    temp = temp[['Hour','TARGET','T-Td','DHI','DNI','WS','RH','T','Time','Sun_up']]

    if is_train==True:          
        temp['Target1'] = temp['TARGET'].shift(-48).fillna(method='ffill')   # 다음날의 Target
        temp['Target2'] = temp['TARGET'].shift(-48*2).fillna(method='ffill') # 다다음날의 Target
        temp = temp.dropna()    # 결측값 제거
        return temp.iloc[:-96]  # 뒤에서 이틀은 뺀다. (예측하고자 하는 날짜이기 때문)

    elif is_train == False:
        temp = temp[['Hour','TARGET','T-Td','DHI','DNI','WS','RH','T','Time','Sun_up']]

        return temp.iloc[-48:, :]

df_train = preprocess_data(train)
# print(df_train.shape)   # (52464, 10)
# print(df_train[:48])

#-------------test 함수정의
df_test = []
for i in range(81):
    file_path = 'test/' + str(i) + '.csv'
    temp = pd.read_csv(file_path)
    temp = preprocess_data(temp, is_train=False)
    df_test.append(temp)

df_test = pd.concat(df_test)
test = np.array(df_test)
print(df_test.shape) # (3888, 8)
x_test = df_test.to_numpy()

###=====================================================================
# 시계열 데이터(함수) --> y1, y2 로 나눠 각각 훈련시키기
def split_xy(dataset, time_steps) :
    x, y1, y2 = [],[],[]
    for i in range(len(dataset)) :
        x_end = i + time_steps
        if x_end > len(dataset) :
            break
        tmp_x = dataset[i:x_end, :-2] # ['Hour' ~ 'T']
        tmp_y1 = dataset[x_end-1:x_end,-2] # Target1
        tmp_y2 = dataset[x_end-1:x_end,-1]   # Target2
        x.append(tmp_x)
        y1.append(tmp_y1)
        y2.append(tmp_y2)
    return np.array(x), np.array(y1), np.array(y2)

X = df_train.to_numpy()
# print(X.shape)      # (52464, 10)
x,y1,y2 = split_xy(X,1)
print(x.shape, y1.shape, y2.shape) #(52464, 1, 8) (52464, 1) (52464, 1)

#####===========전처리
x_train, x_val, y1_train, y1_val, y2_train, y2_val = train_test_split(x,y1,y2,
                           train_size = 0.7,shuffle = False, random_state = 0)

# print(x_train.shape,x_val.shape) #(36724, 1, 8) (15740, 1, 8)
print(y1_train.shape, y1_val.shape, y2_train.shape, y2_val.shape) #(36724, 1) (15740, 1) (36724, 1) (15740, 1)
x_train= x_train.reshape(36724*1, 10)
x_val= x_val.reshape(15740*1, 10)

scaler = StandardScaler()
scaler.fit(x_train)
x_train = scaler.transform(x_train)
x_test = scaler.transform(x_test)
x_val = scaler.transform(x_val)

x_train= x_train.reshape(36724, 1, 10)
x_val= x_val.reshape(15740, 1, 10)
x_test= x_test.reshape(3888, 1, 10)
# print(x_train.shape,x_val.shape,x_test.shape) #(36724, 1, 8) (15740, 1, 8) (3888, 1, 8)

# 함수 : Quantile loss definition
def quantile_loss(q, y_true, y_pred):
    err = (y_true - y_pred)
    return K.mean(K.maximum(q*err, (q-1)*err), axis=-1)

q = [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9]

#2. 모델링

def modeling() :
    inputs = Input(shape=(x_train.shape[1], x_train.shape[2]))
    adam = Adam(learning_rate=0.003)
    conv = Conv1D(filters = 256,kernel_size=2,activation='relu',padding='same')(inputs)
    conv = BatchNormalization()(conv)
    conv = Conv1D(filters = 128,kernel_size=2,activation='relu',padding='same')(conv)
    conv = BatchNormalization()(conv)
    conv = Conv1D(filters = 128,kernel_size=2,activation='relu',padding='same')(conv)
    conv = BatchNormalization()(conv)
    conv = Conv1D(filters = 64,kernel_size=2,activation='relu',padding='same')(conv)
    conv = BatchNormalization()(conv)
    conv = Conv1D(filters = 32,kernel_size=2,activation='relu',padding='same')(conv)
    conv = BatchNormalization()(conv)
    conv = Flatten()(conv)

    conv1 = Conv1D(filters = 256,kernel_size=2,activation='relu',padding='same')(inputs)
    conv1 = BatchNormalization()(conv1)
    conv1 = Conv1D(filters = 128,kernel_size=2,activation='relu',padding='same')(conv1)
    conv1 = BatchNormalization()(conv1)
    conv1 = Conv1D(filters = 128,kernel_size=2,activation='relu',padding='same')(conv1)
    conv1 = BatchNormalization()(conv1)
    conv1 = Conv1D(filters = 64,kernel_size=2,activation='relu',padding='same')(conv1)
    conv1 = BatchNormalization()(conv1)
    conv1 = Conv1D(filters = 32,kernel_size=2,activation='relu',padding='same')(conv1)
    conv1 = BatchNormalization()(conv1)
    conv1 = Flatten()(conv1)

    conv2 = Conv1D(filters = 256,kernel_size=3,activation='relu',padding='same')(inputs)
    conv2 = BatchNormalization()(conv2)
    conv2 = Conv1D(filters = 128,kernel_size=3,activation='relu',padding='same')(conv2)
    conv2 = BatchNormalization()(conv2)
    conv2 = Conv1D(filters = 128,kernel_size=3,activation='relu',padding='same')(conv2)
    conv2 = BatchNormalization()(conv2)
    conv2 = Conv1D(filters = 64,kernel_size=3,activation='relu',padding='same')(conv2)
    conv2 = BatchNormalization()(conv2)
    conv2 = Conv1D(filters = 32,kernel_size=3,activation='relu',padding='same')(conv2)
    conv2 = BatchNormalization()(conv2)
    conv2 = Flatten()(conv2)
    
    conv3 = Conv1D(filters = 256,kernel_size=1,activation='relu',padding='same')(inputs)
    conv3 = BatchNormalization()(conv3)
    conv3 = Conv1D(filters = 128,kernel_size=1,activation='relu',padding='same')(conv3)
    conv3 = BatchNormalization()(conv3)
    conv3 = Conv1D(filters = 128,kernel_size=1,activation='relu',padding='same')(conv3)
    conv3 = BatchNormalization()(conv3)
    conv3 = Conv1D(filters = 64,kernel_size=1,activation='relu',padding='same')(conv3)
    conv3 = BatchNormalization()(conv3)
    conv3 = Conv1D(filters = 32,kernel_size=1,activation='relu',padding='same')(conv3)
    conv3 = BatchNormalization()(conv3)
    conv3 = Flatten()(conv3)
    
    lstm = GRU(256,activation='relu')(inputs)
    lstm = LayerNormalization()(lstm)
    concat = concatenate([conv,conv1,conv2,conv3,lstm])
    dense = Dense(128,activation='relu')(concat)
    dense = Dense(64,activation='relu')(dense)
    dense = Dense(32,activation='relu')(dense)
    dense = Dense(8,activation='relu')(dense)
    outputs = Dense(1,activation='relu')(dense)
    model = Model(inputs=inputs,outputs=outputs)
    return model

# 3. 컴파일, 훈련
#####========컴파일,  훈련, predict
#y1, y2 각각 저장하기

from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau, ModelCheckpoint
es = EarlyStopping(monitor = 'val_loss', patience = 10)
lr = ReduceLROnPlateau(monitor = 'val_loss', patience = 5, factor = 0.9, verbose = 1)
epochs = 200
bs = 512






x = []
for i in [0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9]:
    print("\n\n ",i,"\n\n")
    filepath_cp = f'models2/dacon_y1_quantile_{i:.1f}.hdf5'
    model = load_model(filepath_cp,compile=False)
    pred = pd.DataFrame(model.predict(x_test).reshape(3888,1).round(2))
    x.append(pred)
df_temp1 = pd.concat(x, axis = 1)
df_temp1[df_temp1<0] = 0
num_temp1 = df_temp1.to_numpy()
submission.loc[submission.id.str.contains("Day7"), "q_0.1":] = num_temp1

x = []
# 모레!!
for i in q:
    print("\n\n ",i,"\n\n")
    filepath_cp = f'models2/dacon_y2_quantile_{i:.1f}.hdf5'
    model = load_model(filepath_cp,compile=False)
    pred = pd.DataFrame(model.predict(x_test).reshape(3888,1).round(2))
    x.append(pred)
df_temp2 = pd.concat(x, axis = 1)
df_temp2[df_temp2<0] = 0
num_temp2 = df_temp2.to_numpy()
submission.loc[submission.id.str.contains("Day8"), "q_0.1":] = num_temp2
submission.to_csv('models2/upgrade_sub_0125_돌돌려놓고감.csv', index = False)

(3888, 10)
(52464, 1, 10) (52464, 1) (52464, 1)
(36724, 1) (15740, 1) (36724, 1) (15740, 1)


  0.1 




  0.2 




  0.3 




  0.4 




  0.5 




  0.6 




  0.7 




  0.8 




  0.9 




  0.1 




  0.2 




  0.3 




  0.4 




  0.5 




  0.6 




  0.7 




  0.8 




  0.9 




In [ ]:
#####다시 해보자구~~~!!도전!!! >o< ^o^
###T-Td 함수 추가해보기(GHI함수 빼기)

import pandas as pd
import numpy as np
import os
import glob
import random
import warnings
import tensorflow.keras.backend as K
warnings.filterwarnings("ignore")

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.models import Sequential, load_model,Model
from tensorflow.keras.layers import Dense, Conv1D, Dropout, MaxPool1D,Flatten, Reshape, LSTM,Input,concatenate,BatchNormalization
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau
from lightgbm import LGBMRegressor
from tensorflow.keras.optimizers import Adam


# 파일 불러오기
train = pd.read_csv('./train/train.csv')
submission = pd.read_csv('sample_submission.csv')

#1. DATA

# GHI column 추가 함수
def Add_features(data):
    c = 243.12
    b = 17.62
    gamma = (b * (data['T']) / (c + (data['T']))) + np.log(data['RH'] / 100)
    dp = ( c * gamma) / (b - gamma)
    data.insert(1,'Td',dp)
    data.insert(1,'T-Td',data['T']-data['Td'])
    data.insert(1,'GHI',data['DNI']+data['DHI'])

    sunup=[]
    for i in range(len(data)):
        temp = data.iloc[i,-1]
        if temp>0.0:
            sunup.append(1)
        else :
            sunup.append(0)
    data['Sun_up']=sunup

    # Time 추가
    Time=[]
    for i in range(len(data)):
        temp = 60*data.iloc[i,1]+data.iloc[i,2]
        Time.append(temp)
    data['Time']=Time


    return data

# train data column정의
def preprocess_data(data, is_train=True):
    c = 243.12
    b = 17.62
    gamma = (b * (data['T']) / (c + (data['T']))) + np.log(data['RH'] / 100)
    dp = ( c * gamma) / (b - gamma)
    data.insert(1,'Td',dp)
    data.insert(1,'T-Td',data['T']-data['Td'])
    data.insert(1,'GHI',data['DNI']+data['DHI'])
    sunup=[]
    for i in range(len(data)):
        temp = data.iloc[i,-1]
        if temp>0.0:
            sunup.append(1)
        else :
            sunup.append(0)
    data['Sun_up']=sunup
    # Time 추가
    Time=[]
    for i in range(len(data)):
        temp = 60*data.iloc[i,1]+data.iloc[i,2]
        Time.append(temp)
    data['Time']=Time
    temp = data.copy()
    temp = temp[['Hour','TARGET','T-Td','DHI','DNI','WS','RH','T','Time','Sun_up']]

    if is_train==True:          
        temp['Target1'] = temp['TARGET'].shift(-48).fillna(method='ffill')   # 다음날의 Target
        temp['Target2'] = temp['TARGET'].shift(-48*2).fillna(method='ffill') # 다다음날의 Target
        temp = temp.dropna()    # 결측값 제거
        return temp.iloc[:-96]  # 뒤에서 이틀은 뺀다. (예측하고자 하는 날짜이기 때문)

    elif is_train == False:
        temp = temp[['Hour','TARGET','T-Td','DHI','DNI','WS','RH','T','Time','Sun_up']]

        return temp.iloc[-48:, :]

df_train = preprocess_data(train)
# print(df_train.shape)   # (52464, 10)
# print(df_train[:48])

#-------------test 함수정의
df_test = []
for i in range(81):
    file_path = 'test/' + str(i) + '.csv'
    temp = pd.read_csv(file_path)
    temp = preprocess_data(temp, is_train=False)
    df_test.append(temp)

df_test = pd.concat(df_test)
test = np.array(df_test)
print(df_test.shape) # (3888, 8)
x_test = df_test.to_numpy()

###=====================================================================
# 시계열 데이터(함수) --> y1, y2 로 나눠 각각 훈련시키기
def split_xy(dataset, time_steps) :
    x, y1, y2 = [],[],[]
    for i in range(len(dataset)) :
        x_end = i + time_steps
        if x_end > len(dataset) :
            break
        tmp_x = dataset[i:x_end, :-2] # ['Hour' ~ 'T']
        tmp_y1 = dataset[x_end-1:x_end,-2] # Target1
        tmp_y2 = dataset[x_end-1:x_end,-1]   # Target2
        x.append(tmp_x)
        y1.append(tmp_y1)
        y2.append(tmp_y2)
    return np.array(x), np.array(y1), np.array(y2)

X = df_train.to_numpy()
# print(X.shape)      # (52464, 10)
x,y1,y2 = split_xy(X,1)
print(x.shape, y1.shape, y2.shape) #(52464, 1, 8) (52464, 1) (52464, 1)

#####===========전처리
x_train, x_val, y1_train, y1_val, y2_train, y2_val = train_test_split(x,y1,y2,
                           train_size = 0.7,shuffle = False, random_state = 0)

# print(x_train.shape,x_val.shape) #(36724, 1, 8) (15740, 1, 8)
print(y1_train.shape, y1_val.shape, y2_train.shape, y2_val.shape) #(36724, 1) (15740, 1) (36724, 1) (15740, 1)
x_train= x_train.reshape(36724*1, 10)
x_val= x_val.reshape(15740*1, 10)

scaler = StandardScaler()
scaler.fit(x_train)
x_train = scaler.transform(x_train)
x_test = scaler.transform(x_test)
x_val = scaler.transform(x_val)

x_train= x_train.reshape(36724, 1, 10)
x_val= x_val.reshape(15740, 1, 10)
x_test= x_test.reshape(3888, 1, 10)
# print(x_train.shape,x_val.shape,x_test.shape) #(36724, 1, 8) (15740, 1, 8) (3888, 1, 8)

# 함수 : Quantile loss definition
def quantile_loss(q, y_true, y_pred):
    err = (y_true - y_pred)
    return K.mean(K.maximum(q*err, (q-1)*err), axis=-1)

q = [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9]

#2. 모델링

def modeling() :
    inputs = Input(shape=(x_train.shape[1], x_train.shape[2]))
    adam = Adam(learning_rate=0.003)
    conv = Conv1D(filters = 256,kernel_size=2,activation='relu',padding='same')(inputs)
    conv = BatchNormalization()(conv)
    conv = Conv1D(filters = 128,kernel_size=2,activation='relu',padding='same')(conv)
    conv = BatchNormalization()(conv)
    conv = Conv1D(filters = 128,kernel_size=2,activation='relu',padding='same')(conv)
    conv = BatchNormalization()(conv)
    conv = Conv1D(filters = 64,kernel_size=2,activation='relu',padding='same')(conv)
    conv = BatchNormalization()(conv)
    conv = Conv1D(filters = 32,kernel_size=2,activation='relu',padding='same')(conv)
    conv = BatchNormalization()(conv)
    conv = Flatten()(conv)

    conv1 = Conv1D(filters = 256,kernel_size=2,activation='relu',padding='same')(inputs)
    conv1 = BatchNormalization()(conv1)
    conv1 = Conv1D(filters = 128,kernel_size=2,activation='relu',padding='same')(conv1)
    conv1 = BatchNormalization()(conv1)
    conv1 = Conv1D(filters = 128,kernel_size=2,activation='relu',padding='same')(conv1)
    conv1 = BatchNormalization()(conv1)
    conv1 = Conv1D(filters = 64,kernel_size=2,activation='relu',padding='same')(conv1)
    conv1 = BatchNormalization()(conv1)
    conv1 = Conv1D(filters = 32,kernel_size=2,activation='relu',padding='same')(conv1)
    conv1 = BatchNormalization()(conv1)
    conv1 = Flatten()(conv1)

    conv2 = Conv1D(filters = 256,kernel_size=3,activation='relu',padding='same')(inputs)
    conv2 = BatchNormalization()(conv2)
    conv2 = Conv1D(filters = 128,kernel_size=3,activation='relu',padding='same')(conv2)
    conv2 = BatchNormalization()(conv2)
    conv2 = Conv1D(filters = 128,kernel_size=3,activation='relu',padding='same')(conv2)
    conv2 = BatchNormalization()(conv2)
    conv2 = Conv1D(filters = 64,kernel_size=3,activation='relu',padding='same')(conv2)
    conv2 = BatchNormalization()(conv2)
    conv2 = Conv1D(filters = 32,kernel_size=3,activation='relu',padding='same')(conv2)
    conv2 = BatchNormalization()(conv2)
    conv2 = Flatten()(conv2)
    
    conv3 = Conv1D(filters = 256,kernel_size=1,activation='relu',padding='same')(inputs)
    conv3 = BatchNormalization()(conv3)
    conv3 = Conv1D(filters = 128,kernel_size=1,activation='relu',padding='same')(conv3)
    conv3 = BatchNormalization()(conv3)
    conv3 = Conv1D(filters = 128,kernel_size=1,activation='relu',padding='same')(conv3)
    conv3 = BatchNormalization()(conv3)
    conv3 = Conv1D(filters = 64,kernel_size=1,activation='relu',padding='same')(conv3)
    conv3 = BatchNormalization()(conv3)
    conv3 = Conv1D(filters = 32,kernel_size=1,activation='relu',padding='same')(conv3)
    conv3 = BatchNormalization()(conv3)
    conv3 = Flatten()(conv3)
    
    lstm = LSTM(256,activation='relu',return_sequences=True)(inputs)
    lstm = LSTM(64,activation='relu',return_sequences=True)(lstm)
    lstm = LSTM(16,activation='relu')(lstm)
    lstm = LayerNormalization()(lstm)
    concat = concatenate([conv,conv1,conv2,conv3,lstm])
    dense = Dense(128,activation='relu')(concat)
    dense = Dense(64,activation='relu')(dense)
    dense = Dense(32,activation='relu')(dense)
    dense = Dense(8,activation='relu')(dense)
    outputs = Dense(1,activation='relu')(dense)
    model = Model(inputs=inputs,outputs=outputs)
    return model
# 3. 컴파일, 훈련
#####========컴파일,  훈련, predict
#y1, y2 각각 저장하기

from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau, ModelCheckpoint
es = EarlyStopping(monitor = 'val_loss', patience = 30)
lr = ReduceLROnPlateau(monitor = 'val_loss', patience = 20, factor = 0.9, verbose = 1)
epochs = 500
bs = 16

# 내일!!
x = []
for i in q:
    if q!=0.5:
        print("\n\n ",i,"\n\n")
        model = modeling()
        filepath_cp = f'models3/dacon_y1_quantile_{i:.1f}.hdf5'
        cp = ModelCheckpoint(filepath_cp,save_best_only=True,monitor = 'val_loss')
        model.compile(loss = lambda y_true,y_pred: quantile_loss(i,y_true,y_pred), optimizer = 'adam', metrics = [lambda y,y_pred: quantile_loss(i,y,y_pred)])
        model.fit(x_train,y1_train,epochs = epochs, batch_size = bs, validation_data = (x_val,y1_val),callbacks = [es,cp,lr])
        pred = pd.DataFrame(model.predict(x_test).reshape(3888,1).round(2))
        x.append(pred)
    else:
        print("\n\n ",i,"\n\n")
        model = modeling()
        filepath_cp = f'models3/dacon_y1_quantile_{i:.1f}.hdf5'
        cp = ModelCheckpoint(filepath_cp,save_best_only=True,monitor = 'val_loss')
        model.compile(loss = 'mae', optimizer = 'adam')
        model.fit(x_train,y1_train,epochs = epochs, batch_size = bs, validation_data = (x_val,y1_val),callbacks = [es,cp,lr])
        pred = pd.DataFrame(model.predict(x_test).reshape(3888,1).round(2))
        x.append(pred)
df_temp1 = pd.concat(x, axis = 1)
df_temp1[df_temp1<0] = 0
num_temp1 = df_temp1.to_numpy()
submission.loc[submission.id.str.contains("Day7"), "q_0.1":] = num_temp1

x = []
# 모레!!
for i in q:
    if q!=0.5:
        print("\n\n ",i,"\n\n")
        model = modeling()
        filepath_cp = f'models3/dacon_y2_quantile_{i:.1f}.hdf5'
        cp = ModelCheckpoint(filepath_cp,save_best_only=True,monitor = 'val_loss')
        model.compile(loss = lambda y_true,y_pred: quantile_loss(i,y_true,y_pred), optimizer = 'adam', metrics = [lambda y,y_pred: quantile_loss(i,y,y_pred)])
        model.fit(x_train,y2_train,epochs = epochs, batch_size = bs, validation_data = (x_val,y2_val),callbacks = [es,cp,lr])
        pred = pd.DataFrame(model.predict(x_test).reshape(3888,1).round(2))
        x.append(pred)
    else:
        print("\n\n ",i,"\n\n")
        model = modeling()
        filepath_cp = f'models3/dacon_y2_quantile_{i:.1f}.hdf5'
        cp = ModelCheckpoint(filepath_cp,save_best_only=True,monitor = 'val_loss')
        model.compile(loss = 'mae', optimizer = 'adam')
        model.fit(x_train,y2_train,epochs = epochs, batch_size = bs, validation_data = (x_val,y2_val),callbacks = [es,cp,lr])
        pred = pd.DataFrame(model.predict(x_test).reshape(3888,1).round(2))
        x.append(pred)
df_temp2 = pd.concat(x, axis = 1)
df_temp2[df_temp2<0] = 0
num_temp2 = df_temp2.to_numpy()
submission.loc[submission.id.str.contains("Day8"), "q_0.1":] = num_temp2

##??????????????근데 이 지표가 잘됐다는거 평가하는 방법있을까?????????  로스밖에 없어
submission.to_csv('models3/upgrade_sub_0125_돌려놓고감_2.csv', index = False)
##제출후 등수 135  -- 1.94653

(3888, 10)
(52464, 1, 10) (52464, 1) (52464, 1)
(36724, 1) (15740, 1) (36724, 1) (15740, 1)


  0.1 


Epoch 1/500
2296/2296 [==============================] - 46s 20ms/step - loss: 1.4483 - <lambda>: 1.4483 - val_loss: 1.5328 - val_<lambda>: 1.5328
Epoch 2/500
2296/2296 [==============================] - 46s 20ms/step - loss: 1.3980 - <lambda>: 1.3980 - val_loss: 1.5295 - val_<lambda>: 1.5295
Epoch 3/500
2296/2296 [==============================] - 46s 20ms/step - loss: 1.3720 - <lambda>: 1.3720 - val_loss: 1.5466 - val_<lambda>: 1.5466
Epoch 4/500
2296/2296 [==============================] - 45s 20ms/step - loss: 1.3609 - <lambda>: 1.3609 - val_loss: 1.5218 - val_<lambda>: 1.5218
Epoch 5/500
2296/2296 [==============================] - 45s 19ms/step - loss: 1.3594 - <lambda>: 1.3594 - val_loss: 1.5155 - val_<lambda>: 1.5155
Epoch 6/500
2296/2296 [==============================] - 44s 19ms/step - loss: 1.3522 - <lambda>: 1.3522 - val_loss: 1.5503 - val_<lambda>: 1.5503
Epoch 7/500
229

2296/2296 [==============================] - 42s 18ms/step - loss: 2.1804 - <lambda>: 2.1804 - val_loss: 2.5048 - val_<lambda>: 2.5048
Epoch 7/500
2296/2296 [==============================] - 42s 18ms/step - loss: 2.1699 - <lambda>: 2.1699 - val_loss: 2.5190 - val_<lambda>: 2.5190
Epoch 8/500
2296/2296 [==============================] - 43s 19ms/step - loss: 2.1603 - <lambda>: 2.1603 - val_loss: 2.4608 - val_<lambda>: 2.4608
Epoch 9/500
2296/2296 [==============================] - 42s 19ms/step - loss: 2.1491 - <lambda>: 2.1491 - val_loss: 2.4432 - val_<lambda>: 2.4432
Epoch 10/500
2296/2296 [==============================] - 42s 18ms/step - loss: 2.1364 - <lambda>: 2.1364 - val_loss: 2.4997 - val_<lambda>: 2.4997
Epoch 11/500
2296/2296 [==============================] - 42s 18ms/step - loss: 2.1265 - <lambda>: 2.1265 - val_loss: 2.4485 - val_<lambda>: 2.4485
Epoch 12/500
2296/2296 [==============================] - 42s 18ms/step - loss: 2.1158 - <lambda>: 2.1158 - val_loss: 2.4127 - v

2296/2296 [==============================] - 42s 18ms/step - loss: 2.4974 - <lambda>: 2.4974 - val_loss: 2.9348 - val_<lambda>: 2.9348
Epoch 15/500
2296/2296 [==============================] - 42s 18ms/step - loss: 2.4948 - <lambda>: 2.4948 - val_loss: 2.8951 - val_<lambda>: 2.8951
Epoch 16/500
2296/2296 [==============================] - 42s 18ms/step - loss: 2.4838 - <lambda>: 2.4838 - val_loss: 2.9716 - val_<lambda>: 2.9716
Epoch 17/500
2296/2296 [==============================] - 42s 18ms/step - loss: 2.4756 - <lambda>: 2.4756 - val_loss: 2.9750 - val_<lambda>: 2.9750
Epoch 18/500
2296/2296 [==============================] - 43s 19ms/step - loss: 2.4597 - <lambda>: 2.4597 - val_loss: 2.9855 - val_<lambda>: 2.9855
Epoch 19/500
2296/2296 [==============================] - 42s 18ms/step - loss: 2.4500 - <lambda>: 2.4500 - val_loss: 2.9066 - val_<lambda>: 2.9066
Epoch 20/500
2296/2296 [==============================] - 42s 18ms/step - loss: 2.4403 - <lambda>: 2.4403 - val_loss: 2.8941 

In [ ]:
#####다시 해보자구~~~!!도전!!! >o< ^o^
###T-Td 함수 추가해보기(GHI함수 빼기)

import pandas as pd
import numpy as np
import os
import glob
import random
import warnings
import tensorflow.keras.backend as K
warnings.filterwarnings("ignore")

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.models import Sequential, load_model,Model
from tensorflow.keras.layers import Dense, Conv1D, Dropout, MaxPool1D,Flatten, Reshape, LSTM,Input,concatenate,BatchNormalization,LayerNormalization,GRU
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau
from lightgbm import LGBMRegressor
from tensorflow.keras.optimizers import Adam


# 파일 불러오기
train = pd.read_csv('./train/train.csv')
submission = pd.read_csv('sample_submission.csv')

#1. DATA

# GHI column 추가 함수
def Add_features(data):
    c = 243.12
    b = 17.62
    gamma = (b * (data['T']) / (c + (data['T']))) + np.log(data['RH'] / 100)
    dp = ( c * gamma) / (b - gamma)
    data.insert(1,'Td',dp)
    data.insert(1,'T-Td',data['T']-data['Td'])
    data.insert(1,'GHI',data['DNI']+data['DHI'])

    sunup=[]
    for i in range(len(data)):
        temp = data.iloc[i,-1]
        if temp>0.0:
            sunup.append(1)
        else :
            sunup.append(0)
    data['Sun_up']=sunup

    # Time 추가
    Time=[]
    for i in range(len(data)):
        temp = 60*data.iloc[i,1]+data.iloc[i,2]
        Time.append(temp)
    data['Time']=Time


    return data

# train data column정의
def preprocess_data(data, is_train=True):
    c = 243.12
    b = 17.62
    gamma = (b * (data['T']) / (c + (data['T']))) + np.log(data['RH'] / 100)
    dp = ( c * gamma) / (b - gamma)
    data.insert(1,'Td',dp)
    data.insert(1,'T-Td',data['T']-data['Td'])
    data.insert(1,'GHI',data['DNI']+data['DHI'])
    sunup=[]
    for i in range(len(data)):
        temp = data.iloc[i,-1]
        if temp>0.0:
            sunup.append(1)
        else :
            sunup.append(0)
    data['Sun_up']=sunup
    # Time 추가
    Time=[]
    for i in range(len(data)):
        temp = 60*data.iloc[i,1]+data.iloc[i,2]
        Time.append(temp)
    data['Time']=Time
    temp = data.copy()
    temp = temp[['Hour','TARGET','T-Td','DHI','DNI','WS','RH','T','Time','Sun_up']]

    if is_train==True:          
        temp['Target1'] = temp['TARGET'].shift(-48).fillna(method='ffill')   # 다음날의 Target
        temp['Target2'] = temp['TARGET'].shift(-48*2).fillna(method='ffill') # 다다음날의 Target
        temp = temp.dropna()    # 결측값 제거
        return temp.iloc[:-96]  # 뒤에서 이틀은 뺀다. (예측하고자 하는 날짜이기 때문)

    elif is_train == False:
        temp = temp[['Hour','TARGET','T-Td','DHI','DNI','WS','RH','T','Time','Sun_up']]

        return temp.iloc[-48:, :]

df_train = preprocess_data(train)
# print(df_train.shape)   # (52464, 10)
# print(df_train[:48])

#-------------test 함수정의
df_test = []
for i in range(81):
    file_path = 'test/' + str(i) + '.csv'
    temp = pd.read_csv(file_path)
    temp = preprocess_data(temp, is_train=False)
    df_test.append(temp)

df_test = pd.concat(df_test)
test = np.array(df_test)
print(df_test.shape) # (3888, 8)
x_test = df_test.to_numpy()

###=====================================================================
# 시계열 데이터(함수) --> y1, y2 로 나눠 각각 훈련시키기
def split_xy(dataset, time_steps) :
    x, y1, y2 = [],[],[]
    for i in range(len(dataset)) :
        x_end = i + time_steps
        if x_end > len(dataset) :
            break
        tmp_x = dataset[i:x_end, :-2] # ['Hour' ~ 'T']
        tmp_y1 = dataset[x_end-1:x_end,-2] # Target1
        tmp_y2 = dataset[x_end-1:x_end,-1]   # Target2
        x.append(tmp_x)
        y1.append(tmp_y1)
        y2.append(tmp_y2)
    return np.array(x), np.array(y1), np.array(y2)

X = df_train.to_numpy()
# print(X.shape)      # (52464, 10)
x,y1,y2 = split_xy(X,1)
print(x.shape, y1.shape, y2.shape) #(52464, 1, 8) (52464, 1) (52464, 1)

#####===========전처리
x_train, x_val, y1_train, y1_val, y2_train, y2_val = train_test_split(x,y1,y2,
                           train_size = 0.7,shuffle = False, random_state = 0)

# print(x_train.shape,x_val.shape) #(36724, 1, 8) (15740, 1, 8)
print(y1_train.shape, y1_val.shape, y2_train.shape, y2_val.shape) #(36724, 1) (15740, 1) (36724, 1) (15740, 1)
x_train= x_train.reshape(36724*1, 10)
x_val= x_val.reshape(15740*1, 10)

scaler = StandardScaler()
scaler.fit(x_train)
x_train = scaler.transform(x_train)
x_test = scaler.transform(x_test)
x_val = scaler.transform(x_val)

x_train= x_train.reshape(36724, 1, 10)
x_val= x_val.reshape(15740, 1, 10)
x_test= x_test.reshape(3888, 1, 10)
# print(x_train.shape,x_val.shape,x_test.shape) #(36724, 1, 8) (15740, 1, 8) (3888, 1, 8)

# 함수 : Quantile loss definition
def quantile_loss(q, y_true, y_pred):
    err = (y_true - y_pred)
    return K.mean(K.maximum(q*err, (q-1)*err), axis=-1)

q = [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9]

#2. 모델링

def modeling() :
    inputs = Input(shape=(x_train.shape[1], x_train.shape[2]))
    adam = Adam(learning_rate=0.003)
    conv = Conv1D(filters = 256,kernel_size=2,activation='relu',padding='same')(inputs)
    conv = BatchNormalization()(conv)
    conv = Conv1D(filters = 128,kernel_size=2,activation='relu',padding='same')(conv)
    conv = BatchNormalization()(conv)
    conv = Conv1D(filters = 128,kernel_size=2,activation='relu',padding='same')(conv)
    conv = BatchNormalization()(conv)
    conv = Conv1D(filters = 64,kernel_size=2,activation='relu',padding='same')(conv)
    conv = BatchNormalization()(conv)
    conv = Conv1D(filters = 32,kernel_size=2,activation='relu',padding='same')(conv)
    conv = BatchNormalization()(conv)
    conv = Flatten()(conv)

    conv1 = Conv1D(filters = 256,kernel_size=2,activation='relu',padding='same')(inputs)
    conv1 = BatchNormalization()(conv1)
    conv1 = Conv1D(filters = 128,kernel_size=2,activation='relu',padding='same')(conv1)
    conv1 = BatchNormalization()(conv1)
    conv1 = Conv1D(filters = 128,kernel_size=2,activation='relu',padding='same')(conv1)
    conv1 = BatchNormalization()(conv1)
    conv1 = Conv1D(filters = 64,kernel_size=2,activation='relu',padding='same')(conv1)
    conv1 = BatchNormalization()(conv1)
    conv1 = Conv1D(filters = 32,kernel_size=2,activation='relu',padding='same')(conv1)
    conv1 = BatchNormalization()(conv1)
    conv1 = Flatten()(conv1)

    conv2 = Conv1D(filters = 256,kernel_size=3,activation='relu',padding='same')(inputs)
    conv2 = BatchNormalization()(conv2)
    conv2 = Conv1D(filters = 128,kernel_size=3,activation='relu',padding='same')(conv2)
    conv2 = BatchNormalization()(conv2)
    conv2 = Conv1D(filters = 128,kernel_size=3,activation='relu',padding='same')(conv2)
    conv2 = BatchNormalization()(conv2)
    conv2 = Conv1D(filters = 64,kernel_size=3,activation='relu',padding='same')(conv2)
    conv2 = BatchNormalization()(conv2)
    conv2 = Conv1D(filters = 32,kernel_size=3,activation='relu',padding='same')(conv2)
    conv2 = BatchNormalization()(conv2)
    conv2 = Flatten()(conv2)
    
    conv3 = Conv1D(filters = 256,kernel_size=1,activation='relu',padding='same')(inputs)
    conv3 = BatchNormalization()(conv3)
    conv3 = Conv1D(filters = 128,kernel_size=1,activation='relu',padding='same')(conv3)
    conv3 = BatchNormalization()(conv3)
    conv3 = Conv1D(filters = 128,kernel_size=1,activation='relu',padding='same')(conv3)
    conv3 = BatchNormalization()(conv3)
    conv3 = Conv1D(filters = 64,kernel_size=1,activation='relu',padding='same')(conv3)
    conv3 = BatchNormalization()(conv3)
    conv3 = Conv1D(filters = 32,kernel_size=1,activation='relu',padding='same')(conv3)
    conv3 = BatchNormalization()(conv3)
    conv3 = Flatten()(conv3)
    
    lstm = GRU(256,activation='relu')(inputs)
    lstm = LayerNormalization()(lstm)
    concat = concatenate([conv,conv1,conv2,conv3,lstm])
    dense = Dense(128,activation='relu')(concat)
    dense = Dense(64,activation='relu')(dense)
    dense = Dense(32,activation='relu')(dense)
    dense = Dense(8,activation='relu')(dense)
    outputs = Dense(1,activation='relu')(dense)
    model = Model(inputs=inputs,outputs=outputs)
    return model

# 3. 컴파일, 훈련
#####========컴파일,  훈련, predict
#y1, y2 각각 저장하기

from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau, ModelCheckpoint
es = EarlyStopping(monitor = 'val_loss', patience = 10)
lr = ReduceLROnPlateau(monitor = 'val_loss', patience = 5, factor = 0.9, verbose = 1)
epochs = 200
bs = 512






x = []
for i in [0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9]:
    print("\n\n ",i,"\n\n")
    filepath_cp = f'models3/dacon_y1_quantile_{i:.1f}.hdf5'
    model = load_model(filepath_cp,compile=False)
    pred = pd.DataFrame(model.predict(x_test).reshape(3888,1).round(2))
    x.append(pred)
df_temp1 = pd.concat(x, axis = 1)
df_temp1[df_temp1<0] = 0
num_temp1 = df_temp1.to_numpy()
submission.loc[submission.id.str.contains("Day7"), "q_0.1":] = num_temp1

x = []
# 모레!!
for i in q:
    print("\n\n ",i,"\n\n")
    filepath_cp = f'models3/dacon_y2_quantile_{i:.1f}.hdf5'
    model = load_model(filepath_cp,compile=False)
    pred = pd.DataFrame(model.predict(x_test).reshape(3888,1).round(2))
    x.append(pred)
df_temp2 = pd.concat(x, axis = 1)
df_temp2[df_temp2<0] = 0
num_temp2 = df_temp2.to_numpy()
submission.loc[submission.id.str.contains("Day8"), "q_0.1":] = num_temp2
submission.to_csv('models3/upgrade_sub_0125_돌돌려놓고감.csv', index = False)

In [ ]:
#####다시 해보자구~~~!!도전!!! >o< ^o^
###T-Td 함수 추가해보기(GHI함수 빼기)

import pandas as pd
import numpy as np
import os
import glob
import random
import warnings
import tensorflow.keras.backend as K
warnings.filterwarnings("ignore")

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.models import Sequential, load_model,Model
from tensorflow.keras.layers import Dense, Conv1D, Dropout, MaxPool1D,Flatten, Reshape, LSTM,Input,concatenate,BatchNormalization
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau
from lightgbm import LGBMRegressor
from tensorflow.keras.optimizers import Adam


# 파일 불러오기
train = pd.read_csv('./train/train.csv')
submission = pd.read_csv('sample_submission.csv')

#1. DATA

# GHI column 추가 함수
def Add_features(data):
    c = 243.12
    b = 17.62
    gamma = (b * (data['T']) / (c + (data['T']))) + np.log(data['RH'] / 100)
    dp = ( c * gamma) / (b - gamma)
    data.insert(1,'Td',dp)
    data.insert(1,'T-Td',data['T']-data['Td'])
    data.insert(1,'GHI',data['DNI']+data['DHI'])

    sunup=[]
    for i in range(len(data)):
        temp = data.iloc[i,-1]
        if temp>0.0:
            sunup.append(1)
        else :
            sunup.append(0)
    data['Sun_up']=sunup

    # Time 추가
    Time=[]
    for i in range(len(data)):
        temp = 60*data.iloc[i,1]+data.iloc[i,2]
        Time.append(temp)
    data['Time']=Time


    return data

# train data column정의
def preprocess_data(data, is_train=True):
    c = 243.12
    b = 17.62
    gamma = (b * (data['T']) / (c + (data['T']))) + np.log(data['RH'] / 100)
    dp = ( c * gamma) / (b - gamma)
    data.insert(1,'Td',dp)
    data.insert(1,'T-Td',data['T']-data['Td'])
    data.insert(1,'GHI',data['DNI']+data['DHI'])
    sunup=[]
    for i in range(len(data)):
        temp = data.iloc[i,-1]
        if temp>0.0:
            sunup.append(1)
        else :
            sunup.append(0)
    data['Sun_up']=sunup
    # Time 추가
    Time=[]
    for i in range(len(data)):
        temp = 60*data.iloc[i,1]+data.iloc[i,2]
        Time.append(temp)
    data['Time']=Time
    temp = data.copy()
    temp = temp[['Hour','TARGET','T-Td','DHI','DNI','WS','RH','T','Time','Sun_up']]

    if is_train==True:          
        temp['Target1'] = temp['TARGET'].shift(-48).fillna(method='ffill')   # 다음날의 Target
        temp['Target2'] = temp['TARGET'].shift(-48*2).fillna(method='ffill') # 다다음날의 Target
        temp = temp.dropna()    # 결측값 제거
        return temp.iloc[:-96]  # 뒤에서 이틀은 뺀다. (예측하고자 하는 날짜이기 때문)

    elif is_train == False:
        temp = temp[['Hour','TARGET','T-Td','DHI','DNI','WS','RH','T','Time','Sun_up']]

        return temp.iloc[-48:, :]

df_train = preprocess_data(train)
# print(df_train.shape)   # (52464, 10)
# print(df_train[:48])

#-------------test 함수정의
df_test = []
for i in range(81):
    file_path = 'test/' + str(i) + '.csv'
    temp = pd.read_csv(file_path)
    temp = preprocess_data(temp, is_train=False)
    df_test.append(temp)

df_test = pd.concat(df_test)
test = np.array(df_test)
print(df_test.shape) # (3888, 8)
x_test = df_test.to_numpy()

###=====================================================================
# 시계열 데이터(함수) --> y1, y2 로 나눠 각각 훈련시키기
def split_xy(dataset, time_steps) :
    x, y1, y2 = [],[],[]
    for i in range(len(dataset)) :
        x_end = i + time_steps
        if x_end > len(dataset) :
            break
        tmp_x = dataset[i:x_end, :-2] # ['Hour' ~ 'T']
        tmp_y1 = dataset[x_end-1:x_end,-2] # Target1
        tmp_y2 = dataset[x_end-1:x_end,-1]   # Target2
        x.append(tmp_x)
        y1.append(tmp_y1)
        y2.append(tmp_y2)
    return np.array(x), np.array(y1), np.array(y2)

X = df_train.to_numpy()
# print(X.shape)      # (52464, 10)
x,y1,y2 = split_xy(X,1)
print(x.shape, y1.shape, y2.shape) #(52464, 1, 8) (52464, 1) (52464, 1)

#####===========전처리
x_train, x_val, y1_train, y1_val, y2_train, y2_val = train_test_split(x,y1,y2,
                           train_size = 0.7,shuffle = False, random_state = 0)

# print(x_train.shape,x_val.shape) #(36724, 1, 8) (15740, 1, 8)
print(y1_train.shape, y1_val.shape, y2_train.shape, y2_val.shape) #(36724, 1) (15740, 1) (36724, 1) (15740, 1)
x_train= x_train.reshape(36724*1, 10)
x_val= x_val.reshape(15740*1, 10)

scaler = StandardScaler()
scaler.fit(x_train)
x_train = scaler.transform(x_train)
x_test = scaler.transform(x_test)
x_val = scaler.transform(x_val)

x_train= x_train.reshape(36724, 1, 10)
x_val= x_val.reshape(15740, 1, 10)
x_test= x_test.reshape(3888, 1, 10)
# print(x_train.shape,x_val.shape,x_test.shape) #(36724, 1, 8) (15740, 1, 8) (3888, 1, 8)

# 함수 : Quantile loss definition
def quantile_loss(q, y_true, y_pred):
    err = (y_true - y_pred)
    return K.mean(K.maximum(q*err, (q-1)*err), axis=-1)

q = [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9]

#2. 모델링

def modeling() :
    inputs = Input(shape=(x_train.shape[1], x_train.shape[2]))
    adam = Adam(learning_rate=0.003)
    conv = Conv1D(filters = 256,kernel_size=2,activation='relu',padding='same')(inputs)
    conv = BatchNormalization()(conv)
    conv = Conv1D(filters = 128,kernel_size=2,activation='relu',padding='same')(conv)
    conv = BatchNormalization()(conv)
    conv = Conv1D(filters = 128,kernel_size=2,activation='relu',padding='same')(conv)
    conv = BatchNormalization()(conv)
    conv = Conv1D(filters = 64,kernel_size=2,activation='relu',padding='same')(conv)
    conv = BatchNormalization()(conv)
    conv = Conv1D(filters = 32,kernel_size=2,activation='relu',padding='same')(conv)
    conv = BatchNormalization()(conv)
    conv = Flatten()(conv)

    conv1 = Conv1D(filters = 256,kernel_size=2,activation='relu',padding='same')(inputs)
    conv1 = BatchNormalization()(conv1)
    conv1 = Conv1D(filters = 128,kernel_size=2,activation='relu',padding='same')(conv1)
    conv1 = BatchNormalization()(conv1)
    conv1 = Conv1D(filters = 128,kernel_size=2,activation='relu',padding='same')(conv1)
    conv1 = BatchNormalization()(conv1)
    conv1 = Conv1D(filters = 64,kernel_size=2,activation='relu',padding='same')(conv1)
    conv1 = BatchNormalization()(conv1)
    conv1 = Conv1D(filters = 32,kernel_size=2,activation='relu',padding='same')(conv1)
    conv1 = BatchNormalization()(conv1)
    conv1 = Flatten()(conv1)

    conv2 = Conv1D(filters = 256,kernel_size=3,activation='relu',padding='same')(inputs)
    conv2 = BatchNormalization()(conv2)
    conv2 = Conv1D(filters = 128,kernel_size=3,activation='relu',padding='same')(conv2)
    conv2 = BatchNormalization()(conv2)
    conv2 = Conv1D(filters = 128,kernel_size=3,activation='relu',padding='same')(conv2)
    conv2 = BatchNormalization()(conv2)
    conv2 = Conv1D(filters = 64,kernel_size=3,activation='relu',padding='same')(conv2)
    conv2 = BatchNormalization()(conv2)
    conv2 = Conv1D(filters = 32,kernel_size=3,activation='relu',padding='same')(conv2)
    conv2 = BatchNormalization()(conv2)
    conv2 = Flatten()(conv2)
    
    conv3 = Conv1D(filters = 256,kernel_size=1,activation='relu',padding='same')(inputs)
    conv3 = BatchNormalization()(conv3)
    conv3 = Conv1D(filters = 128,kernel_size=1,activation='relu',padding='same')(conv3)
    conv3 = BatchNormalization()(conv3)
    conv3 = Conv1D(filters = 128,kernel_size=1,activation='relu',padding='same')(conv3)
    conv3 = BatchNormalization()(conv3)
    conv3 = Conv1D(filters = 64,kernel_size=1,activation='relu',padding='same')(conv3)
    conv3 = BatchNormalization()(conv3)
    conv3 = Conv1D(filters = 32,kernel_size=1,activation='relu',padding='same')(conv3)
    conv3 = BatchNormalization()(conv3)
    conv3 = Flatten()(conv3)
    
    lstm = LSTM(256,activation='relu',return_sequences=True)(inputs)
    lstm = LSTM(64,activation='relu',return_sequences=True)(lstm)
    lstm = LSTM(16,activation='relu')(lstm)
    lstm = LayerNormalization()(lstm)
    
    lstm1 = LSTM(256,activation='relu',return_sequences=True)(inputs)
    lstm1 = LSTM(16,activation='relu')(lstm1)
    lstm1 = LayerNormalization()(lstm1)
    
    lstm2 = LSTM(16,activation='relu')(inputs)
    lstm2 = LayerNormalization()(lstm2)
    
    
    concat = concatenate([conv,conv1,conv2,conv3,lstm,lstm1,lstm2])
    dense = Dense(128,activation='relu')(concat)
    dense = Dense(64,activation='relu')(dense)
    dense = Dense(32,activation='relu')(dense)
    dense = Dense(8,activation='relu')(dense)
    outputs = Dense(1,activation='relu')(dense)
    model = Model(inputs=inputs,outputs=outputs)
    return model
# 3. 컴파일, 훈련
#####========컴파일,  훈련, predict
#y1, y2 각각 저장하기

from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau, ModelCheckpoint
es = EarlyStopping(monitor = 'val_loss', patience = 30)
lr = ReduceLROnPlateau(monitor = 'val_loss', patience = 20, factor = 0.9, verbose = 1)
epochs = 500
bs = 16

# 내일!!
x = []
for i in q:
    if q!=0.5:
        print("\n\n ",i,"\n\n")
        model = modeling()
        filepath_cp = f'models4/dacon_y1_quantile_{i:.1f}.hdf5'
        cp = ModelCheckpoint(filepath_cp,save_best_only=True,monitor = 'val_loss')
        model.compile(loss = lambda y_true,y_pred: quantile_loss(i,y_true,y_pred), optimizer = 'adam', metrics = [lambda y,y_pred: quantile_loss(i,y,y_pred)])
        model.fit(x_train,y1_train,epochs = epochs, batch_size = bs, validation_data = (x_val,y1_val),callbacks = [es,cp,lr])
        pred = pd.DataFrame(model.predict(x_test).reshape(3888,1).round(2))
        x.append(pred)
    else:
        print("\n\n ",i,"\n\n")
        model = modeling()
        filepath_cp = f'models4/dacon_y1_quantile_{i:.1f}.hdf5'
        cp = ModelCheckpoint(filepath_cp,save_best_only=True,monitor = 'val_loss')
        model.compile(loss = 'mae', optimizer = 'adam')
        model.fit(x_train,y1_train,epochs = epochs, batch_size = bs, validation_data = (x_val,y1_val),callbacks = [es,cp,lr])
        pred = pd.DataFrame(model.predict(x_test).reshape(3888,1).round(2))
        x.append(pred)
df_temp1 = pd.concat(x, axis = 1)
df_temp1[df_temp1<0] = 0
num_temp1 = df_temp1.to_numpy()
submission.loc[submission.id.str.contains("Day7"), "q_0.1":] = num_temp1

x = []
# 모레!!
for i in q:
    if q!=0.5:
        print("\n\n ",i,"\n\n")
        model = modeling()
        filepath_cp = f'models4/dacon_y2_quantile_{i:.1f}.hdf5'
        cp = ModelCheckpoint(filepath_cp,save_best_only=True,monitor = 'val_loss')
        model.compile(loss = lambda y_true,y_pred: quantile_loss(i,y_true,y_pred), optimizer = 'adam', metrics = [lambda y,y_pred: quantile_loss(i,y,y_pred)])
        model.fit(x_train,y2_train,epochs = epochs, batch_size = bs, validation_data = (x_val,y2_val),callbacks = [es,cp,lr])
        pred = pd.DataFrame(model.predict(x_test).reshape(3888,1).round(2))
        x.append(pred)
    else:
        print("\n\n ",i,"\n\n")
        model = modeling()
        filepath_cp = f'models4/dacon_y2_quantile_{i:.1f}.hdf5'
        cp = ModelCheckpoint(filepath_cp,save_best_only=True,monitor = 'val_loss')
        model.compile(loss = 'mae', optimizer = 'adam')
        model.fit(x_train,y2_train,epochs = epochs, batch_size = bs, validation_data = (x_val,y2_val),callbacks = [es,cp,lr])
        pred = pd.DataFrame(model.predict(x_test).reshape(3888,1).round(2))
        x.append(pred)
df_temp2 = pd.concat(x, axis = 1)
df_temp2[df_temp2<0] = 0
num_temp2 = df_temp2.to_numpy()
submission.loc[submission.id.str.contains("Day8"), "q_0.1":] = num_temp2

##??????????????근데 이 지표가 잘됐다는거 평가하는 방법있을까?????????  로스밖에 없어
submission.to_csv('models4/upgrade_sub_0125_돌려놓고감_2.csv', index = False)
##제출후 등수 135  -- 1.94653

In [ ]:
#####다시 해보자구~~~!!도전!!! >o< ^o^
###T-Td 함수 추가해보기(GHI함수 빼기)

import pandas as pd
import numpy as np
import os
import glob
import random
import warnings
import tensorflow.keras.backend as K
warnings.filterwarnings("ignore")

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.models import Sequential, load_model,Model
from tensorflow.keras.layers import Dense, Conv1D, Dropout, MaxPool1D,Flatten, Reshape, LSTM,Input,concatenate,BatchNormalization,LayerNormalization,GRU
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau
from lightgbm import LGBMRegressor
from tensorflow.keras.optimizers import Adam


# 파일 불러오기
train = pd.read_csv('./train/train.csv')
submission = pd.read_csv('sample_submission.csv')

#1. DATA

# GHI column 추가 함수
def Add_features(data):
    c = 243.12
    b = 17.62
    gamma = (b * (data['T']) / (c + (data['T']))) + np.log(data['RH'] / 100)
    dp = ( c * gamma) / (b - gamma)
    data.insert(1,'Td',dp)
    data.insert(1,'T-Td',data['T']-data['Td'])
    data.insert(1,'GHI',data['DNI']+data['DHI'])

    sunup=[]
    for i in range(len(data)):
        temp = data.iloc[i,-1]
        if temp>0.0:
            sunup.append(1)
        else :
            sunup.append(0)
    data['Sun_up']=sunup

    # Time 추가
    Time=[]
    for i in range(len(data)):
        temp = 60*data.iloc[i,1]+data.iloc[i,2]
        Time.append(temp)
    data['Time']=Time


    return data

# train data column정의
def preprocess_data(data, is_train=True):
    c = 243.12
    b = 17.62
    gamma = (b * (data['T']) / (c + (data['T']))) + np.log(data['RH'] / 100)
    dp = ( c * gamma) / (b - gamma)
    data.insert(1,'Td',dp)
    data.insert(1,'T-Td',data['T']-data['Td'])
    data.insert(1,'GHI',data['DNI']+data['DHI'])
    sunup=[]
    for i in range(len(data)):
        temp = data.iloc[i,-1]
        if temp>0.0:
            sunup.append(1)
        else :
            sunup.append(0)
    data['Sun_up']=sunup
    # Time 추가
    Time=[]
    for i in range(len(data)):
        temp = 60*data.iloc[i,1]+data.iloc[i,2]
        Time.append(temp)
    data['Time']=Time
    temp = data.copy()
    temp = temp[['Hour','TARGET','T-Td','DHI','DNI','WS','RH','T','Time','Sun_up']]

    if is_train==True:          
        temp['Target1'] = temp['TARGET'].shift(-48).fillna(method='ffill')   # 다음날의 Target
        temp['Target2'] = temp['TARGET'].shift(-48*2).fillna(method='ffill') # 다다음날의 Target
        temp = temp.dropna()    # 결측값 제거
        return temp.iloc[:-96]  # 뒤에서 이틀은 뺀다. (예측하고자 하는 날짜이기 때문)

    elif is_train == False:
        temp = temp[['Hour','TARGET','T-Td','DHI','DNI','WS','RH','T','Time','Sun_up']]

        return temp.iloc[-48:, :]

df_train = preprocess_data(train)
# print(df_train.shape)   # (52464, 10)
# print(df_train[:48])

#-------------test 함수정의
df_test = []
for i in range(81):
    file_path = 'test/' + str(i) + '.csv'
    temp = pd.read_csv(file_path)
    temp = preprocess_data(temp, is_train=False)
    df_test.append(temp)

df_test = pd.concat(df_test)
test = np.array(df_test)
print(df_test.shape) # (3888, 8)
x_test = df_test.to_numpy()

###=====================================================================
# 시계열 데이터(함수) --> y1, y2 로 나눠 각각 훈련시키기
def split_xy(dataset, time_steps) :
    x, y1, y2 = [],[],[]
    for i in range(len(dataset)) :
        x_end = i + time_steps
        if x_end > len(dataset) :
            break
        tmp_x = dataset[i:x_end, :-2] # ['Hour' ~ 'T']
        tmp_y1 = dataset[x_end-1:x_end,-2] # Target1
        tmp_y2 = dataset[x_end-1:x_end,-1]   # Target2
        x.append(tmp_x)
        y1.append(tmp_y1)
        y2.append(tmp_y2)
    return np.array(x), np.array(y1), np.array(y2)

X = df_train.to_numpy()
# print(X.shape)      # (52464, 10)
x,y1,y2 = split_xy(X,1)
print(x.shape, y1.shape, y2.shape) #(52464, 1, 8) (52464, 1) (52464, 1)

#####===========전처리
x_train, x_val, y1_train, y1_val, y2_train, y2_val = train_test_split(x,y1,y2,
                           train_size = 0.7,shuffle = False, random_state = 0)

# print(x_train.shape,x_val.shape) #(36724, 1, 8) (15740, 1, 8)
print(y1_train.shape, y1_val.shape, y2_train.shape, y2_val.shape) #(36724, 1) (15740, 1) (36724, 1) (15740, 1)
x_train= x_train.reshape(36724*1, 10)
x_val= x_val.reshape(15740*1, 10)

scaler = StandardScaler()
scaler.fit(x_train)
x_train = scaler.transform(x_train)
x_test = scaler.transform(x_test)
x_val = scaler.transform(x_val)

x_train= x_train.reshape(36724, 1, 10)
x_val= x_val.reshape(15740, 1, 10)
x_test= x_test.reshape(3888, 1, 10)
# print(x_train.shape,x_val.shape,x_test.shape) #(36724, 1, 8) (15740, 1, 8) (3888, 1, 8)

# 함수 : Quantile loss definition
def quantile_loss(q, y_true, y_pred):
    err = (y_true - y_pred)
    return K.mean(K.maximum(q*err, (q-1)*err), axis=-1)

q = [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9]

#2. 모델링

def modeling() :
    inputs = Input(shape=(x_train.shape[1], x_train.shape[2]))
    adam = Adam(learning_rate=0.003)
    conv = Conv1D(filters = 256,kernel_size=2,activation='relu',padding='same')(inputs)
    conv = BatchNormalization()(conv)
    conv = Conv1D(filters = 128,kernel_size=2,activation='relu',padding='same')(conv)
    conv = BatchNormalization()(conv)
    conv = Conv1D(filters = 128,kernel_size=2,activation='relu',padding='same')(conv)
    conv = BatchNormalization()(conv)
    conv = Conv1D(filters = 64,kernel_size=2,activation='relu',padding='same')(conv)
    conv = BatchNormalization()(conv)
    conv = Conv1D(filters = 32,kernel_size=2,activation='relu',padding='same')(conv)
    conv = BatchNormalization()(conv)
    conv = Flatten()(conv)

    conv1 = Conv1D(filters = 256,kernel_size=2,activation='relu',padding='same')(inputs)
    conv1 = BatchNormalization()(conv1)
    conv1 = Conv1D(filters = 128,kernel_size=2,activation='relu',padding='same')(conv1)
    conv1 = BatchNormalization()(conv1)
    conv1 = Conv1D(filters = 128,kernel_size=2,activation='relu',padding='same')(conv1)
    conv1 = BatchNormalization()(conv1)
    conv1 = Conv1D(filters = 64,kernel_size=2,activation='relu',padding='same')(conv1)
    conv1 = BatchNormalization()(conv1)
    conv1 = Conv1D(filters = 32,kernel_size=2,activation='relu',padding='same')(conv1)
    conv1 = BatchNormalization()(conv1)
    conv1 = Flatten()(conv1)

    conv2 = Conv1D(filters = 256,kernel_size=3,activation='relu',padding='same')(inputs)
    conv2 = BatchNormalization()(conv2)
    conv2 = Conv1D(filters = 128,kernel_size=3,activation='relu',padding='same')(conv2)
    conv2 = BatchNormalization()(conv2)
    conv2 = Conv1D(filters = 128,kernel_size=3,activation='relu',padding='same')(conv2)
    conv2 = BatchNormalization()(conv2)
    conv2 = Conv1D(filters = 64,kernel_size=3,activation='relu',padding='same')(conv2)
    conv2 = BatchNormalization()(conv2)
    conv2 = Conv1D(filters = 32,kernel_size=3,activation='relu',padding='same')(conv2)
    conv2 = BatchNormalization()(conv2)
    conv2 = Flatten()(conv2)
    
    conv3 = Conv1D(filters = 256,kernel_size=1,activation='relu',padding='same')(inputs)
    conv3 = BatchNormalization()(conv3)
    conv3 = Conv1D(filters = 128,kernel_size=1,activation='relu',padding='same')(conv3)
    conv3 = BatchNormalization()(conv3)
    conv3 = Conv1D(filters = 128,kernel_size=1,activation='relu',padding='same')(conv3)
    conv3 = BatchNormalization()(conv3)
    conv3 = Conv1D(filters = 64,kernel_size=1,activation='relu',padding='same')(conv3)
    conv3 = BatchNormalization()(conv3)
    conv3 = Conv1D(filters = 32,kernel_size=1,activation='relu',padding='same')(conv3)
    conv3 = BatchNormalization()(conv3)
    conv3 = Flatten()(conv3)
    
    lstm = GRU(256,activation='relu')(inputs)
    lstm = LayerNormalization()(lstm)
    concat = concatenate([conv,conv1,conv2,conv3,lstm])
    dense = Dense(128,activation='relu')(concat)
    dense = Dense(64,activation='relu')(dense)
    dense = Dense(32,activation='relu')(dense)
    dense = Dense(8,activation='relu')(dense)
    outputs = Dense(1,activation='relu')(dense)
    model = Model(inputs=inputs,outputs=outputs)
    return model

# 3. 컴파일, 훈련
#####========컴파일,  훈련, predict
#y1, y2 각각 저장하기

from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau, ModelCheckpoint
es = EarlyStopping(monitor = 'val_loss', patience = 10)
lr = ReduceLROnPlateau(monitor = 'val_loss', patience = 5, factor = 0.9, verbose = 1)
epochs = 200
bs = 512






x = []
for i in [0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9]:
    print("\n\n ",i,"\n\n")
    filepath_cp = f'models4/dacon_y1_quantile_{i:.1f}.hdf5'
    model = load_model(filepath_cp,compile=False)
    pred = pd.DataFrame(model.predict(x_test).reshape(3888,1).round(2))
    x.append(pred)
df_temp1 = pd.concat(x, axis = 1)
df_temp1[df_temp1<0] = 0
num_temp1 = df_temp1.to_numpy()
submission.loc[submission.id.str.contains("Day7"), "q_0.1":] = num_temp1

x = []
# 모레!!
for i in q:
    print("\n\n ",i,"\n\n")
    filepath_cp = f'models4/dacon_y2_quantile_{i:.1f}.hdf5'
    model = load_model(filepath_cp,compile=False)
    pred = pd.DataFrame(model.predict(x_test).reshape(3888,1).round(2))
    x.append(pred)
df_temp2 = pd.concat(x, axis = 1)
df_temp2[df_temp2<0] = 0
num_temp2 = df_temp2.to_numpy()
submission.loc[submission.id.str.contains("Day8"), "q_0.1":] = num_temp2
submission.to_csv('models4/upgrade_sub_0125_돌돌려놓고감.csv', index = False)

In [ ]:
#####다시 해보자구~~~!!도전!!! >o< ^o^
###T-Td 함수 추가해보기(GHI함수 빼기)

import pandas as pd
import numpy as np
import os
import glob
import random
import warnings
import tensorflow.keras.backend as K
warnings.filterwarnings("ignore")

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.models import Sequential, load_model,Model
from tensorflow.keras.layers import Dense, Conv1D, Dropout, MaxPool1D,Flatten, Reshape, LSTM,Input,concatenate,BatchNormalization
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau
from lightgbm import LGBMRegressor
from tensorflow.keras.optimizers import Adam


# 파일 불러오기
train = pd.read_csv('./train/train.csv')
submission = pd.read_csv('sample_submission.csv')

#1. DATA

# GHI column 추가 함수
def Add_features(data):
    c = 243.12
    b = 17.62
    gamma = (b * (data['T']) / (c + (data['T']))) + np.log(data['RH'] / 100)
    dp = ( c * gamma) / (b - gamma)
    data.insert(1,'Td',dp)
    data.insert(1,'T-Td',data['T']-data['Td'])
    data.insert(1,'GHI',data['DNI']+data['DHI'])

    sunup=[]
    for i in range(len(data)):
        temp = data.iloc[i,-1]
        if temp>0.0:
            sunup.append(1)
        else :
            sunup.append(0)
    data['Sun_up']=sunup

    # Time 추가
    Time=[]
    for i in range(len(data)):
        temp = 60*data.iloc[i,1]+data.iloc[i,2]
        Time.append(temp)
    data['Time']=Time


    return data

# train data column정의
def preprocess_data(data, is_train=True):
    c = 243.12
    b = 17.62
    gamma = (b * (data['T']) / (c + (data['T']))) + np.log(data['RH'] / 100)
    dp = ( c * gamma) / (b - gamma)
    data.insert(1,'Td',dp)
    data.insert(1,'T-Td',data['T']-data['Td'])
    data.insert(1,'GHI',data['DNI']+data['DHI'])
    sunup=[]
    for i in range(len(data)):
        temp = data.iloc[i,-1]
        if temp>0.0:
            sunup.append(1)
        else :
            sunup.append(0)
    data['Sun_up']=sunup
    # Time 추가
    Time=[]
    for i in range(len(data)):
        temp = 60*data.iloc[i,1]+data.iloc[i,2]
        Time.append(temp)
    data['Time']=Time
    temp = data.copy()
    temp = temp[['Hour','TARGET','T-Td','DHI','DNI','WS','RH','T','Time','Sun_up']]

    if is_train==True:          
        temp['Target1'] = temp['TARGET'].shift(-48).fillna(method='ffill')   # 다음날의 Target
        temp['Target2'] = temp['TARGET'].shift(-48*2).fillna(method='ffill') # 다다음날의 Target
        temp = temp.dropna()    # 결측값 제거
        return temp.iloc[:-96]  # 뒤에서 이틀은 뺀다. (예측하고자 하는 날짜이기 때문)

    elif is_train == False:
        temp = temp[['Hour','TARGET','T-Td','DHI','DNI','WS','RH','T','Time','Sun_up']]

        return temp.iloc[-48:, :]

df_train = preprocess_data(train)
# print(df_train.shape)   # (52464, 10)
# print(df_train[:48])

#-------------test 함수정의
df_test = []
for i in range(81):
    file_path = 'test/' + str(i) + '.csv'
    temp = pd.read_csv(file_path)
    temp = preprocess_data(temp, is_train=False)
    df_test.append(temp)

df_test = pd.concat(df_test)
test = np.array(df_test)
print(df_test.shape) # (3888, 8)
x_test = df_test.to_numpy()

###=====================================================================
# 시계열 데이터(함수) --> y1, y2 로 나눠 각각 훈련시키기
def split_xy(dataset, time_steps) :
    x, y1, y2 = [],[],[]
    for i in range(len(dataset)) :
        x_end = i + time_steps
        if x_end > len(dataset) :
            break
        tmp_x = dataset[i:x_end, :-2] # ['Hour' ~ 'T']
        tmp_y1 = dataset[x_end-1:x_end,-2] # Target1
        tmp_y2 = dataset[x_end-1:x_end,-1]   # Target2
        x.append(tmp_x)
        y1.append(tmp_y1)
        y2.append(tmp_y2)
    return np.array(x), np.array(y1), np.array(y2)

X = df_train.to_numpy()
# print(X.shape)      # (52464, 10)
x,y1,y2 = split_xy(X,1)
print(x.shape, y1.shape, y2.shape) #(52464, 1, 8) (52464, 1) (52464, 1)

#####===========전처리
x_train, x_val, y1_train, y1_val, y2_train, y2_val = train_test_split(x,y1,y2,
                           train_size = 0.7,shuffle = False, random_state = 0)

# print(x_train.shape,x_val.shape) #(36724, 1, 8) (15740, 1, 8)
print(y1_train.shape, y1_val.shape, y2_train.shape, y2_val.shape) #(36724, 1) (15740, 1) (36724, 1) (15740, 1)
x_train= x_train.reshape(36724*1, 10)
x_val= x_val.reshape(15740*1, 10)

scaler = StandardScaler()
scaler.fit(x_train)
x_train = scaler.transform(x_train)
x_test = scaler.transform(x_test)
x_val = scaler.transform(x_val)

x_train= x_train.reshape(36724, 1, 10)
x_val= x_val.reshape(15740, 1, 10)
x_test= x_test.reshape(3888, 1, 10)
# print(x_train.shape,x_val.shape,x_test.shape) #(36724, 1, 8) (15740, 1, 8) (3888, 1, 8)

# 함수 : Quantile loss definition
def quantile_loss(q, y_true, y_pred):
    err = (y_true - y_pred)
    return K.mean(K.maximum(q*err, (q-1)*err), axis=-1)

q = [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9]

#2. 모델링

def modeling() :
    inputs = Input(shape=(x_train.shape[1], x_train.shape[2]))
    adam = Adam(learning_rate=0.003)
    conv = Conv1D(filters = 256,kernel_size=2,activation='relu',padding='same')(inputs)
    conv = BatchNormalization()(conv)
    conv = Conv1D(filters = 128,kernel_size=2,activation='relu',padding='same')(conv)
    conv = BatchNormalization()(conv)
    conv = Conv1D(filters = 128,kernel_size=2,activation='relu',padding='same')(conv)
    conv = BatchNormalization()(conv)
    conv = Conv1D(filters = 64,kernel_size=2,activation='relu',padding='same')(conv)
    conv = BatchNormalization()(conv)
    conv = Conv1D(filters = 32,kernel_size=2,activation='relu',padding='same')(conv)
    conv = BatchNormalization()(conv)
    conv = Flatten()(conv)

    conv1 = Conv1D(filters = 256,kernel_size=2,activation='relu',padding='same')(inputs)
    conv1 = BatchNormalization()(conv1)
    conv1 = Conv1D(filters = 128,kernel_size=2,activation='relu',padding='same')(conv1)
    conv1 = BatchNormalization()(conv1)
    conv1 = Conv1D(filters = 128,kernel_size=2,activation='relu',padding='same')(conv1)
    conv1 = BatchNormalization()(conv1)
    conv1 = Conv1D(filters = 64,kernel_size=2,activation='relu',padding='same')(conv1)
    conv1 = BatchNormalization()(conv1)
    conv1 = Conv1D(filters = 32,kernel_size=2,activation='relu',padding='same')(conv1)
    conv1 = BatchNormalization()(conv1)
    conv1 = LSTM(32,activation='relu')(conv1)

    conv2 = Conv1D(filters = 256,kernel_size=3,activation='relu',padding='same')(inputs)
    conv2 = BatchNormalization()(conv2)
    conv2 = Conv1D(filters = 128,kernel_size=3,activation='relu',padding='same')(conv2)
    conv2 = BatchNormalization()(conv2)
    conv2 = Conv1D(filters = 128,kernel_size=3,activation='relu',padding='same')(conv2)
    conv2 = BatchNormalization()(conv2)
    conv2 = Conv1D(filters = 64,kernel_size=3,activation='relu',padding='same')(conv2)
    conv2 = BatchNormalization()(conv2)
    conv2 = Conv1D(filters = 32,kernel_size=3,activation='relu',padding='same')(conv2)
    conv2 = BatchNormalization()(conv2)
    conv2 = LSTM(32,activation='relu')(conv2)
    
    conv3 = Conv1D(filters = 256,kernel_size=1,activation='relu',padding='same')(inputs)
    conv3 = BatchNormalization()(conv3)
    conv3 = Conv1D(filters = 128,kernel_size=1,activation='relu',padding='same')(conv3)
    conv3 = BatchNormalization()(conv3)
    conv3 = Conv1D(filters = 128,kernel_size=1,activation='relu',padding='same')(conv3)
    conv3 = BatchNormalization()(conv3)
    conv3 = Conv1D(filters = 64,kernel_size=1,activation='relu',padding='same')(conv3)
    conv3 = BatchNormalization()(conv3)
    conv3 = Conv1D(filters = 32,kernel_size=1,activation='relu',padding='same')(conv3)
    conv3 = BatchNormalization()(conv3)
    conv3 = LSTM(32,activation='relu')(conv3)
    
    lstm = LSTM(256,activation='relu',return_sequences=True)(inputs)
    lstm = LSTM(64,activation='relu',return_sequences=True)(lstm)
    lstm = LSTM(16,activation='relu')(lstm)
    lstm = LayerNormalization()(lstm)
    
    lstm1 = LSTM(256,activation='relu',return_sequences=True)(inputs)
    lstm1 = LSTM(16,activation='relu')(lstm1)
    lstm1 = LayerNormalization()(lstm1)
    
    lstm2 = LSTM(16,activation='relu')(inputs)
    lstm2 = LayerNormalization()(lstm2)
    
    concat = concatenate([conv,conv1,conv2,conv3,lstm,lstm1,lstm2])
    dense = Dense(128,activation='relu')(concat)
    dense = Dense(64,activation='relu')(dense)
    dense = Dense(32,activation='relu')(dense)
    dense = Dense(8,activation='relu')(dense)
    outputs = Dense(1,activation='relu')(dense)
    model = Model(inputs=inputs,outputs=outputs)
    return model
# 3. 컴파일, 훈련
#####========컴파일,  훈련, predict
#y1, y2 각각 저장하기

from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau, ModelCheckpoint
es = EarlyStopping(monitor = 'val_loss', patience = 30)
lr = ReduceLROnPlateau(monitor = 'val_loss', patience = 20, factor = 0.9, verbose = 1)
epochs = 500
bs = 16

# 내일!!
x = []
for i in q:
    if q!=0.5:
        print("\n\n ",i,"\n\n")
        model = modeling()
        filepath_cp = f'models5/dacon_y1_quantile_{i:.1f}.hdf5'
        cp = ModelCheckpoint(filepath_cp,save_best_only=True,monitor = 'val_loss')
        model.compile(loss = lambda y_true,y_pred: quantile_loss(i,y_true,y_pred), optimizer = 'adam', metrics = [lambda y,y_pred: quantile_loss(i,y,y_pred)])
        model.fit(x_train,y1_train,epochs = epochs, batch_size = bs, validation_data = (x_val,y1_val),callbacks = [es,cp,lr])
        pred = pd.DataFrame(model.predict(x_test).reshape(3888,1).round(2))
        x.append(pred)
    else:
        print("\n\n ",i,"\n\n")
        model = modeling()
        filepath_cp = f'models5/dacon_y1_quantile_{i:.1f}.hdf5'
        cp = ModelCheckpoint(filepath_cp,save_best_only=True,monitor = 'val_loss')
        model.compile(loss = 'mae', optimizer = 'adam')
        model.fit(x_train,y1_train,epochs = epochs, batch_size = bs, validation_data = (x_val,y1_val),callbacks = [es,cp,lr])
        pred = pd.DataFrame(model.predict(x_test).reshape(3888,1).round(2))
        x.append(pred)
df_temp1 = pd.concat(x, axis = 1)
df_temp1[df_temp1<0] = 0
num_temp1 = df_temp1.to_numpy()
submission.loc[submission.id.str.contains("Day7"), "q_0.1":] = num_temp1

x = []
# 모레!!
for i in q:
    if q!=0.5:
        print("\n\n ",i,"\n\n")
        model = modeling()
        filepath_cp = f'models5/dacon_y2_quantile_{i:.1f}.hdf5'
        cp = ModelCheckpoint(filepath_cp,save_best_only=True,monitor = 'val_loss')
        model.compile(loss = lambda y_true,y_pred: quantile_loss(i,y_true,y_pred), optimizer = 'adam', metrics = [lambda y,y_pred: quantile_loss(i,y,y_pred)])
        model.fit(x_train,y2_train,epochs = epochs, batch_size = bs, validation_data = (x_val,y2_val),callbacks = [es,cp,lr])
        pred = pd.DataFrame(model.predict(x_test).reshape(3888,1).round(2))
        x.append(pred)
    else:
        print("\n\n ",i,"\n\n")
        model = modeling()
        filepath_cp = f'models5/dacon_y2_quantile_{i:.1f}.hdf5'
        cp = ModelCheckpoint(filepath_cp,save_best_only=True,monitor = 'val_loss')
        model.compile(loss = 'mae', optimizer = 'adam')
        model.fit(x_train,y2_train,epochs = epochs, batch_size = bs, validation_data = (x_val,y2_val),callbacks = [es,cp,lr])
        pred = pd.DataFrame(model.predict(x_test).reshape(3888,1).round(2))
        x.append(pred)
df_temp2 = pd.concat(x, axis = 1)
df_temp2[df_temp2<0] = 0
num_temp2 = df_temp2.to_numpy()
submission.loc[submission.id.str.contains("Day8"), "q_0.1":] = num_temp2

##??????????????근데 이 지표가 잘됐다는거 평가하는 방법있을까?????????  로스밖에 없어
submission.to_csv('models5/upgrade_sub_0125_돌려놓고감_2.csv', index = False)
##제출후 등수 135  -- 1.94653

In [ ]:
#####다시 해보자구~~~!!도전!!! >o< ^o^
###T-Td 함수 추가해보기(GHI함수 빼기)

import pandas as pd
import numpy as np
import os
import glob
import random
import warnings
import tensorflow.keras.backend as K
warnings.filterwarnings("ignore")

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.models import Sequential, load_model,Model
from tensorflow.keras.layers import Dense, Conv1D, Dropout, MaxPool1D,Flatten, Reshape, LSTM,Input,concatenate,BatchNormalization,LayerNormalization,GRU
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau
from lightgbm import LGBMRegressor
from tensorflow.keras.optimizers import Adam


# 파일 불러오기
train = pd.read_csv('./train/train.csv')
submission = pd.read_csv('sample_submission.csv')

#1. DATA

# GHI column 추가 함수
def Add_features(data):
    c = 243.12
    b = 17.62
    gamma = (b * (data['T']) / (c + (data['T']))) + np.log(data['RH'] / 100)
    dp = ( c * gamma) / (b - gamma)
    data.insert(1,'Td',dp)
    data.insert(1,'T-Td',data['T']-data['Td'])
    data.insert(1,'GHI',data['DNI']+data['DHI'])

    sunup=[]
    for i in range(len(data)):
        temp = data.iloc[i,-1]
        if temp>0.0:
            sunup.append(1)
        else :
            sunup.append(0)
    data['Sun_up']=sunup

    # Time 추가
    Time=[]
    for i in range(len(data)):
        temp = 60*data.iloc[i,1]+data.iloc[i,2]
        Time.append(temp)
    data['Time']=Time


    return data

# train data column정의
def preprocess_data(data, is_train=True):
    c = 243.12
    b = 17.62
    gamma = (b * (data['T']) / (c + (data['T']))) + np.log(data['RH'] / 100)
    dp = ( c * gamma) / (b - gamma)
    data.insert(1,'Td',dp)
    data.insert(1,'T-Td',data['T']-data['Td'])
    data.insert(1,'GHI',data['DNI']+data['DHI'])
    sunup=[]
    for i in range(len(data)):
        temp = data.iloc[i,-1]
        if temp>0.0:
            sunup.append(1)
        else :
            sunup.append(0)
    data['Sun_up']=sunup
    # Time 추가
    Time=[]
    for i in range(len(data)):
        temp = 60*data.iloc[i,1]+data.iloc[i,2]
        Time.append(temp)
    data['Time']=Time
    temp = data.copy()
    temp = temp[['Hour','TARGET','T-Td','DHI','DNI','WS','RH','T','Time','Sun_up']]

    if is_train==True:          
        temp['Target1'] = temp['TARGET'].shift(-48).fillna(method='ffill')   # 다음날의 Target
        temp['Target2'] = temp['TARGET'].shift(-48*2).fillna(method='ffill') # 다다음날의 Target
        temp = temp.dropna()    # 결측값 제거
        return temp.iloc[:-96]  # 뒤에서 이틀은 뺀다. (예측하고자 하는 날짜이기 때문)

    elif is_train == False:
        temp = temp[['Hour','TARGET','T-Td','DHI','DNI','WS','RH','T','Time','Sun_up']]

        return temp.iloc[-48:, :]

df_train = preprocess_data(train)
# print(df_train.shape)   # (52464, 10)
# print(df_train[:48])

#-------------test 함수정의
df_test = []
for i in range(81):
    file_path = 'test/' + str(i) + '.csv'
    temp = pd.read_csv(file_path)
    temp = preprocess_data(temp, is_train=False)
    df_test.append(temp)

df_test = pd.concat(df_test)
test = np.array(df_test)
print(df_test.shape) # (3888, 8)
x_test = df_test.to_numpy()

###=====================================================================
# 시계열 데이터(함수) --> y1, y2 로 나눠 각각 훈련시키기
def split_xy(dataset, time_steps) :
    x, y1, y2 = [],[],[]
    for i in range(len(dataset)) :
        x_end = i + time_steps
        if x_end > len(dataset) :
            break
        tmp_x = dataset[i:x_end, :-2] # ['Hour' ~ 'T']
        tmp_y1 = dataset[x_end-1:x_end,-2] # Target1
        tmp_y2 = dataset[x_end-1:x_end,-1]   # Target2
        x.append(tmp_x)
        y1.append(tmp_y1)
        y2.append(tmp_y2)
    return np.array(x), np.array(y1), np.array(y2)

X = df_train.to_numpy()
# print(X.shape)      # (52464, 10)
x,y1,y2 = split_xy(X,1)
print(x.shape, y1.shape, y2.shape) #(52464, 1, 8) (52464, 1) (52464, 1)

#####===========전처리
x_train, x_val, y1_train, y1_val, y2_train, y2_val = train_test_split(x,y1,y2,
                           train_size = 0.7,shuffle = False, random_state = 0)

# print(x_train.shape,x_val.shape) #(36724, 1, 8) (15740, 1, 8)
print(y1_train.shape, y1_val.shape, y2_train.shape, y2_val.shape) #(36724, 1) (15740, 1) (36724, 1) (15740, 1)
x_train= x_train.reshape(36724*1, 10)
x_val= x_val.reshape(15740*1, 10)

scaler = StandardScaler()
scaler.fit(x_train)
x_train = scaler.transform(x_train)
x_test = scaler.transform(x_test)
x_val = scaler.transform(x_val)

x_train= x_train.reshape(36724, 1, 10)
x_val= x_val.reshape(15740, 1, 10)
x_test= x_test.reshape(3888, 1, 10)
# print(x_train.shape,x_val.shape,x_test.shape) #(36724, 1, 8) (15740, 1, 8) (3888, 1, 8)

# 함수 : Quantile loss definition
def quantile_loss(q, y_true, y_pred):
    err = (y_true - y_pred)
    return K.mean(K.maximum(q*err, (q-1)*err), axis=-1)

q = [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9]

#2. 모델링

def modeling() :
    inputs = Input(shape=(x_train.shape[1], x_train.shape[2]))
    adam = Adam(learning_rate=0.003)
    conv = Conv1D(filters = 256,kernel_size=2,activation='relu',padding='same')(inputs)
    conv = BatchNormalization()(conv)
    conv = Conv1D(filters = 128,kernel_size=2,activation='relu',padding='same')(conv)
    conv = BatchNormalization()(conv)
    conv = Conv1D(filters = 128,kernel_size=2,activation='relu',padding='same')(conv)
    conv = BatchNormalization()(conv)
    conv = Conv1D(filters = 64,kernel_size=2,activation='relu',padding='same')(conv)
    conv = BatchNormalization()(conv)
    conv = Conv1D(filters = 32,kernel_size=2,activation='relu',padding='same')(conv)
    conv = BatchNormalization()(conv)
    conv = Flatten()(conv)

    conv1 = Conv1D(filters = 256,kernel_size=2,activation='relu',padding='same')(inputs)
    conv1 = BatchNormalization()(conv1)
    conv1 = Conv1D(filters = 128,kernel_size=2,activation='relu',padding='same')(conv1)
    conv1 = BatchNormalization()(conv1)
    conv1 = Conv1D(filters = 128,kernel_size=2,activation='relu',padding='same')(conv1)
    conv1 = BatchNormalization()(conv1)
    conv1 = Conv1D(filters = 64,kernel_size=2,activation='relu',padding='same')(conv1)
    conv1 = BatchNormalization()(conv1)
    conv1 = Conv1D(filters = 32,kernel_size=2,activation='relu',padding='same')(conv1)
    conv1 = BatchNormalization()(conv1)
    conv1 = Flatten()(conv1)

    conv2 = Conv1D(filters = 256,kernel_size=3,activation='relu',padding='same')(inputs)
    conv2 = BatchNormalization()(conv2)
    conv2 = Conv1D(filters = 128,kernel_size=3,activation='relu',padding='same')(conv2)
    conv2 = BatchNormalization()(conv2)
    conv2 = Conv1D(filters = 128,kernel_size=3,activation='relu',padding='same')(conv2)
    conv2 = BatchNormalization()(conv2)
    conv2 = Conv1D(filters = 64,kernel_size=3,activation='relu',padding='same')(conv2)
    conv2 = BatchNormalization()(conv2)
    conv2 = Conv1D(filters = 32,kernel_size=3,activation='relu',padding='same')(conv2)
    conv2 = BatchNormalization()(conv2)
    conv2 = Flatten()(conv2)
    
    conv3 = Conv1D(filters = 256,kernel_size=1,activation='relu',padding='same')(inputs)
    conv3 = BatchNormalization()(conv3)
    conv3 = Conv1D(filters = 128,kernel_size=1,activation='relu',padding='same')(conv3)
    conv3 = BatchNormalization()(conv3)
    conv3 = Conv1D(filters = 128,kernel_size=1,activation='relu',padding='same')(conv3)
    conv3 = BatchNormalization()(conv3)
    conv3 = Conv1D(filters = 64,kernel_size=1,activation='relu',padding='same')(conv3)
    conv3 = BatchNormalization()(conv3)
    conv3 = Conv1D(filters = 32,kernel_size=1,activation='relu',padding='same')(conv3)
    conv3 = BatchNormalization()(conv3)
    conv3 = Flatten()(conv3)
    
    lstm = GRU(256,activation='relu')(inputs)
    lstm = LayerNormalization()(lstm)
    concat = concatenate([conv,conv1,conv2,conv3,lstm])
    dense = Dense(128,activation='relu')(concat)
    dense = Dense(64,activation='relu')(dense)
    dense = Dense(32,activation='relu')(dense)
    dense = Dense(8,activation='relu')(dense)
    outputs = Dense(1,activation='relu')(dense)
    model = Model(inputs=inputs,outputs=outputs)
    return model

# 3. 컴파일, 훈련
#####========컴파일,  훈련, predict
#y1, y2 각각 저장하기

from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau, ModelCheckpoint
es = EarlyStopping(monitor = 'val_loss', patience = 10)
lr = ReduceLROnPlateau(monitor = 'val_loss', patience = 5, factor = 0.9, verbose = 1)
epochs = 200
bs = 512






x = []
for i in [0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9]:
    print("\n\n ",i,"\n\n")
    filepath_cp = f'models5/dacon_y1_quantile_{i:.1f}.hdf5'
    model = load_model(filepath_cp,compile=False)
    pred = pd.DataFrame(model.predict(x_test).reshape(3888,1).round(2))
    x.append(pred)
df_temp1 = pd.concat(x, axis = 1)
df_temp1[df_temp1<0] = 0
num_temp1 = df_temp1.to_numpy()
submission.loc[submission.id.str.contains("Day7"), "q_0.1":] = num_temp1

x = []
# 모레!!
for i in q:
    print("\n\n ",i,"\n\n")
    filepath_cp = f'models5/dacon_y2_quantile_{i:.1f}.hdf5'
    model = load_model(filepath_cp,compile=False)
    pred = pd.DataFrame(model.predict(x_test).reshape(3888,1).round(2))
    x.append(pred)
df_temp2 = pd.concat(x, axis = 1)
df_temp2[df_temp2<0] = 0
num_temp2 = df_temp2.to_numpy()
submission.loc[submission.id.str.contains("Day8"), "q_0.1":] = num_temp2
submission.to_csv('models5/upgrade_sub_0125_돌돌려놓고감.csv', index = False)